In [ ]:
#!/usr/bin/env python3
"""
pip_package.py (fixed)
──────────────────────
• Dynamic rolling-window PIP trader (no look-ahead).
• Back-test harness that writes KPI CSV & PNG equity plot.
• No hard-wired assumption about instrument count.
"""
from pathlib import Path
from typing  import List, Optional, Tuple, Dict, Callable

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# ═════════════ infrastructure stubs ═════════════
def export(fn: Callable) -> Callable:
    fn._is_exported = True
    return fn


class Trader:
    def Alg(self, prcSoFar: np.ndarray) -> np.ndarray:
        raise NotImplementedError


# ═════════════ strategy hyper-params (grid-search output) ═════════════
BEST_N = [
    200, 200, 400, 200, 400, 400, 100, 300, 100, 200,
    100, 500, 100, 300, 200, 100, 200, 100, 200, 300,
    100, 200, 500, 100, 400, 100, 100, 400, 100, 100,
    400, 500, 300, 200, 400, 500, 100, 400, 100, 100,
    100, 300, 400, 500, 100, 400, 500, 100, 500, 300,
]
BEST_W = [
     10,  10, 100,  10, 100,  10, 100,  25,  10, 100,
    100,  50,  25,  10,  50,  25,  10,  25,  50, 100,
    100,  10,  10,  50,  25,  25,  10,  50,  25, 100,
     25,  10, 100,  50,  25,  10,  50, 100,  50,  25,
     25,  10, 100, 100,  50,  50,  10,  25,  25,  25,
]

FALLBACK_N, FALLBACK_W = 200, 25        # if price set has >50 instruments
POSITION_LIMIT         = 10_000
THR_MIN, THR_MAX, THR_STEP = 0.0, 0.700, 0.0005
THR_GRID               = np.arange(THR_MIN, THR_MAX + THR_STEP, THR_STEP)
COMM_RATE              = 0.0005

# ═════════════ helpers ═════════════
def _causal_update(px: float,
                   last_extreme: float,
                   direction: int,
                   thr: float) -> Tuple[int, float]:
    if direction == 0:
        move = (px - last_extreme) / last_extreme if last_extreme else 0.0
        if abs(move) >= thr:
            return (1 if move > 0 else -1), px
        return 0, last_extreme

    if direction == 1:
        if px > last_extreme:
            return 1, px
        if (last_extreme - px) / last_extreme >= thr:
            return -1, px
        return 1, last_extreme

    if px < last_extreme:
        return -1, px
    if (px - last_extreme) / last_extreme >= thr:
        return 1, px
    return -1, last_extreme


def _best_threshold(window: np.ndarray) -> float:
    if len(window) < 10:        # window too small → fallback
        return 0.05
    best_thr, best_pnl = THR_GRID[0], -np.inf
    for thr in THR_GRID:
        dir_vec = np.zeros(len(window), np.int8)
        le, d = window[0], 0
        for k in range(1, len(window)):
            d, le = _causal_update(window[k], le, d, thr)
            dir_vec[k] = d
        pos = dir_vec.astype(np.int32) * POSITION_LIMIT
        pnl = (pos[:-1] * np.diff(window)).sum()
        if pnl > best_pnl:
            best_pnl, best_thr = pnl, thr
    return best_thr


# ═════════════ strategy class ═════════════
class CausalPIPTrader(Trader):

    class _State:
        __slots__ = ("n","w","next_cal","thr","dir","le")
        def __init__(self, n:int, w:int):
            self.n, self.w    = n, w
            self.next_cal     = n
            self.thr          = None
            self.dir          = 0
            self.le           = None

    def __init__(self, max_instruments: int | None = None):
        super().__init__()
        self._states: List[CausalPIPTrader._State] = []
        self._max_inst = max_instruments
        print("CausalPIPTrader initialised.")

    def _ensure_states(self, n_inst: int):
        """Expand state list if price file has more instruments than grid."""
        while len(self._states) < n_inst:
            idx = len(self._states)
            if idx < len(BEST_N):
                n, w = BEST_N[idx], BEST_W[idx]
            else:                          # fallback for overflow symbols
                n, w = FALLBACK_N, FALLBACK_W
            self._states.append(self._State(n, w))

    @export
    def Alg(self, prcSoFar: np.ndarray) -> np.ndarray:
        n_inst, t_seen = prcSoFar.shape
        if self._max_inst and n_inst > self._max_inst:
            raise ValueError("Instrument count exceeded limit.")
        self._ensure_states(n_inst)

        positions = np.zeros(n_inst, np.int32)

        for i in range(n_inst):
            st    = self._states[i]
            price = prcSoFar[i, -1]

            if st.le is None:       # init last extreme
                st.le = price

            if t_seen >= st.next_cal:
                win_start = max(0, t_seen - st.n)
                st.thr    = _best_threshold(prcSoFar[i, win_start:t_seen])
                st.next_cal += st.w
                print(f"[t={t_seen:4d}] Inst {i:02d} new thr={st.thr:.4f}")

            if st.thr is None:
                continue

            st.dir, st.le = _causal_update(price, st.le, st.dir, st.thr)
            positions[i]  = st.dir * POSITION_LIMIT

        return positions


# ═════════════ back-test harness ═════════════
def run_backtest(prices_path: str = "prices.txt") -> pd.DataFrame:
    prices = np.loadtxt(Path(prices_path), delimiter=None)
    T, nI  = prices.shape
    tr     = CausalPIPTrader()
    rec = {"inst":[], "bar":[], "price":[], "pos":[], "pnl":[], "equity":[]}
    pos_prev = np.zeros(nI, np.int32)
    equity   = np.zeros(nI, np.float64)

    for t in range(1, T):
        pos = tr.Alg(prices[:t+1, :].T)   
        delta = pos - pos_prev
        pnl   = pos_prev * (prices[t] - prices[t-1])
        comm  = COMM_RATE * np.abs(delta) * prices[t]
        equity += pnl - comm

        for i in range(nI):
            rec["inst"].append(i)
            rec["bar"].append(t)
            rec["price"].append(prices[t, i])
            rec["pos"].append(pos[i])
            rec["pnl"].append(pnl[i]-comm[i])
            rec["equity"].append(equity[i])

        pos_prev = pos.copy()

    return pd.DataFrame(rec)


def _kpis(df: pd.DataFrame) -> pd.DataFrame:
    def agg(sub):
        tot = sub.pnl.sum()
        vol = sub.pnl.std(ddof=0)
        sharpe = tot / vol * np.sqrt(252*6.5*60) if vol else 0
        dd = (sub.equity.cummax() - sub.equity).max()
        return pd.Series({"TotalPnL": tot, "Sharpe": sharpe, "MaxDD": dd})
    return df.groupby("inst").apply(agg).reset_index()


def _plot_bottom(df: pd.DataFrame, kpi: pd.DataFrame, out: str):
    worst = kpi.nsmallest(max(1, len(kpi)//4), "Sharpe").inst
    plt.figure(figsize=(12,6))
    for i in worst:
        sub = df[df.inst == i]
        plt.plot(sub.bar, sub.equity, label=f"Inst {i:02d}", alpha=.6)
    plt.title("Bottom quartile equity curves")
    plt.legend(ncol=4, fontsize=7)
    plt.tight_layout()
    plt.savefig(out, dpi=150)
    plt.close()


# ═════════════ CLI run ═════════════
if __name__ == "__main__":
    OUT_CSV = "pip_kpis.csv"
    OUT_PNG = "pip_equity.png"

    df  = run_backtest("prices.txt")
    kpi = _kpis(df)
    kpi.to_csv(OUT_CSV, index=False)
    _plot_bottom(df, kpi, OUT_PNG)

    print(kpi.head(10).to_string(index=False))
    print(f"\nSaved KPI table → {OUT_CSV}")
    print(f"Saved equity plot → {OUT_PNG}")


In [ ]:
#!/usr/bin/env python3
"""
weakness_analysis.py
─────────────────────
Runs the CausalPIPTrader on prices.txt, records detailed per-bar metrics,
and analyzes where / why the model underperforms by examining:

  • Volatility regimes (rolling σ of returns)
  • Threshold bands (quartiles of state.thr)
  • Trade types (new swing vs continuation)
  • Instrument-level drawdowns

At the end, prints summaries of the worst conditions and exports
a CSV of annotated bar-level results.
"""
from pathlib import Path
from typing    import List, Optional, Tuple, Dict, Callable

import numpy as np
import pandas as pd

# ─────────────────── infrastructure stubs ────────────────────────
def export(fn: Callable) -> Callable:
    fn._is_exported = True
    return fn

class Trader:
    def Alg(self, prcSoFar: np.ndarray) -> np.ndarray:
        raise NotImplementedError

# ───────────────── strategy hyper-params (from grid-search) ──────────
BEST_N = [
    200, 200, 400, 200, 400, 400, 100, 300, 100, 200,
    100, 500, 100, 300, 200, 100, 200, 100, 200, 300,
    100, 200, 500, 100, 400, 100, 100, 400, 100, 100,
    400, 500, 300, 200, 400, 500, 100, 400, 100, 100,
    100, 300, 400, 500, 100, 400, 500, 100, 500, 300,
]
BEST_W = [
     10,  10, 100,  10, 100,  10, 100,  25,  10, 100,
    100,  50,  25,  10,  50,  25,  10,  25,  50, 100,
    100,  10,  10,  50,  25,  25,  10,  50,  25, 100,
     25,  10, 100,  50,  25,  10,  50, 100,  50,  25,
     25,  10, 100, 100,  50,  50,  10,  25,  25,  25,
]
FALLBACK_N, FALLBACK_W = 200, 25
POSITION_LIMIT        = 10_000
THR_MIN, THR_MAX, THR_STEP = 0.0, 0.700, 0.0005
THR_GRID              = np.arange(THR_MIN, THR_MAX+THR_STEP, THR_STEP)
COMM_RATE             = 0.0005

# ─────────────────── helper functions ────────────────────────────
def _causal_update(px: float,
                   last_extreme: float,
                   direction: int,
                   thr: float) -> Tuple[int,float]:
    if direction == 0:
        move = (px - last_extreme)/last_extreme if last_extreme else 0.0
        if abs(move) >= thr:
            return (1 if move>0 else -1), px
        return 0, last_extreme
    if direction == 1:
        if px > last_extreme: return 1, px
        if (last_extreme-px)/last_extreme >= thr: return -1, px
        return 1, last_extreme
    # direction == -1
    if px < last_extreme: return -1, px
    if (px-last_extreme)/last_extreme >= thr: return 1, px
    return -1, last_extreme

def _best_threshold(window: np.ndarray) -> float:
    if len(window)<10: return 0.05
    best_thr, best_pnl = THR_GRID[0], -np.inf
    for thr in THR_GRID:
        le, d = window[0], 0
        pnl = 0.0
        for px0, px1 in zip(window[:-1], window[1:]):
            d, le = _causal_update(px1, le, d, thr)
            pnl += d*POSITION_LIMIT*(px1-px0)
        if pnl>best_pnl:
            best_pnl, best_thr = pnl, thr
    return best_thr

# ────────────────── trader class ───────────────────────────────
class CausalPIPTrader(Trader):
    class _State:
        __slots__ = ("n","w","next_cal","thr","dir","le")
        def __init__(self,n,w):
            self.n, self.w = n,w
            self.next_cal  = n
            self.thr       = None
            self.dir       = 0
            self.le        = None

    def __init__(self):
        super().__init__()
        self._states: List[CausalPIPTrader._State] = []
    def _ensure(self,n_inst:int):
        while len(self._states)<n_inst:
            idx = len(self._states)
            n,w = (BEST_N[idx],BEST_W[idx]) if idx<len(BEST_N) else (FALLBACK_N,FALLBACK_W)
            self._states.append(self._State(n,w))

    @export
    def Alg(self, prcSoFar: np.ndarray) -> np.ndarray:
        n_inst, t_seen = prcSoFar.shape
        self._ensure(n_inst)
        pos = np.zeros(n_inst, np.int32)
        for i in range(n_inst):
            st = self._states[i]
            px = prcSoFar[i,-1]
            if st.le is None: st.le=px
            if t_seen>=st.next_cal:
                win = prcSoFar[i, t_seen-st.n:t_seen]
                st.thr = _best_threshold(win)
                st.next_cal += st.w
            if st.thr is None:
                continue
            st.dir, st.le = _causal_update(px, st.le, st.dir, st.thr)
            pos[i] = st.dir*POSITION_LIMIT
        return pos

# ────────────────── backtest + analysis ──────────────────────────
def run_and_analyze(prices_path="prices.txt"):
    prices = np.loadtxt(Path(prices_path), delimiter=None)
    T,nI   = prices.shape
    trader = CausalPIPTrader()

    rec: Dict[str,List] = {
        "inst":[],"bar":[],"price":[],"vol":[],"thr":[],
        "dir":[],"pos":[],"pnl":[],"equity":[]
    }
    pos_prev = np.zeros(nI, np.int32)
    equity   = np.zeros(nI, np.float64)
    # rolling volatility buffer
    ret_buf = np.zeros((50,nI), np.float64)
    for t in range(1,T):
        # update returns buffer
        ret_buf[t%50] = (prices[t]-prices[t-1])/prices[t-1]
        vol = np.nanstd(ret_buf, axis=0)
        # run strategy
        pos = trader.Alg(prices[:t+1,:].T)
        delta = pos-pos_prev
        pnl   = pos_prev*(prices[t]-prices[t-1])
        comm  = COMM_RATE*np.abs(delta)*prices[t]
        equity += pnl-comm
        for i,st in enumerate(trader._states):
            rec["inst"].append(i)
            rec["bar"].append(t)
            rec["price"].append(prices[t,i])
            rec["vol"].append(vol[i])
            rec["thr"].append(st.thr if st.thr is not None else np.nan)
            rec["dir"].append(st.dir)
            rec["pos"].append(pos[i])
            rec["pnl"].append(pnl[i]-comm[i])
            rec["equity"].append(equity[i])
        pos_prev = pos.copy()

    df = pd.DataFrame(rec)
    # overall stats
    print("\n=== Overall PnL by Volatility Regime ===")
    df["vol_q"] = pd.qcut(df.vol, 4, labels=False)
    vol_grp = df.groupby("vol_q").pnl.agg(['mean','sum','count'])
    print(vol_grp)

    print("\n=== PnL by Threshold Quartile ===")
    df["thr_q"] = pd.qcut(df.thr, 4, labels=False, duplicates='drop')
    thr_grp = df.groupby("thr_q").pnl.agg(['mean','sum','count'])
    print(thr_grp)

    print("\n=== Trade Type Analysis ===")
    df["is_new_swing"] = (df.dir.diff()!=0)
    type_grp = df.groupby("is_new_swing").pnl.agg(['mean','sum','count'])
    print(type_grp)

    print("\n=== Worst Instruments ===")
    inst_grp = df.groupby("inst").pnl.sum().nsmallest(5)
    print(inst_grp)

    # export annotated data
    df.to_csv("detailed_results.csv", index=False)
    print("\nDetailed per-bar results -> detailed_results.csv")

if __name__=="__main__":
    run_and_analyze()


In [ ]:
#!/usr/bin/env python3
"""
compare_filters.py
──────────────────
1.  Runs the current “black-list / overflow-safe” PIP trader **as is**.
2.  Runs an **enhanced** version that:
        • never trades if 5-bar momentum is between the 25- and 75-
          percentile of the full data set (momentum Q1-Q2 bucket);
        • never trades on the *first* bar of a new swing
          (swing-length == 1 ⇒ swing-length Q0 bucket).

Prints side-by-side portfolio metrics for both runs:
    Total PnL, mean per-bar PnL, σ, Sharpe (raw $).

Only highlights the results; no files are written.
"""
from pathlib import Path
from typing   import Optional, Tuple, List, Dict

import numpy as np
import pandas as pd

# =============================== parameters ===============================
BLACK_LIST = {4, 5, 8, 22, 23, 28, 48, 49}
POSITION_LIMIT = 10_000
COMM_RATE      = 0.0005
THR_MIN, THR_MAX, THR_STEP = 0.0, 0.700, 0.0005
THR_GRID = np.arange(THR_MIN, THR_MAX + THR_STEP, THR_STEP)

# (N,W) pairs from grid-search
BEST_N = [
    200,200,400,200,400,400,100,300,100,200,100,500,100,300,200,100,200,100,200,300,
    100,200,500,100,400,100,100,400,100,100,400,500,300,200,400,500,100,400,100,100,
    100,300,400,500,100,400,500,100,500,300,
]
BEST_W = [
     10, 10,100, 10,100, 10,100, 25, 10,100,100, 50, 25, 10, 50, 25, 10, 25, 50,100,
    100, 10, 10, 50, 25, 25, 10, 50, 25,100, 25, 10,100, 50, 25, 10, 50,100, 50, 25,
     25, 10,100,100, 50, 50, 10, 25, 25, 25,
]

# =========================== helper functions =============================
def causal_update(px: float, le: float, d: int, thr: float) -> Tuple[int, float]:
    """One-step pivot update."""
    if d == 0:
        move = (px - le) / le
        if abs(move) >= thr:
            return (1 if move > 0 else -1), px
        return 0, le
    if d == 1:
        if px > le: return 1, px
        if (le - px) / le >= thr: return -1, px
        return 1, le
    # d == -1
    if px < le: return -1, px
    if (px - le) / le >= thr: return 1, px
    return -1, le


def best_threshold(window: np.ndarray) -> float:
    best_thr, best_pnl = THR_GRID[0], -np.inf
    for thr in THR_GRID:
        d, le = 0, window[0]
        pnl = 0.0
        for p0, p1 in zip(window[:-1], window[1:]):
            d, le = causal_update(p1, le, d, thr)
            pnl += d * POSITION_LIMIT * (p1 - p0)
        if pnl > best_pnl:
            best_pnl, best_thr = pnl, thr
    return best_thr


# ====================== base trader (unchanged) ===========================
class BasePIPTrader(Trader):
    class _S: __slots__ = ("n","w","next","thr","dir","le")
    def __init__(self):
        self._st = [self._S() for _ in range(50)]
        for s,n,w in zip(self._st, BEST_N, BEST_W):
            s.n, s.w, s.next, s.thr, s.dir, s.le = n, w, n, None, 0, None
    @export
    def Alg(self, prc: np.ndarray) -> np.ndarray:
        n,t = prc.shape
        pos = np.zeros(n, np.int32)
        for i,s in enumerate(self._st):
            if i in BLACK_LIST: continue
            px = prc[i,-1]
            if s.le is None: s.le = px
            if t >= s.next:
                win = prc[i, t-s.n:t]
                s.thr  = best_threshold(win)
                s.next = t + s.w
            if s.thr is None: continue
            s.dir, s.le = causal_update(px, s.le, s.dir, s.thr)
            pos[i] = s.dir * POSITION_LIMIT
        return pos

# ================ enhanced trader with filters ============================
class FilteredPIPTrader(BasePIPTrader):
    """Adds two filters:
       • trade only if |mom5| in outer quartiles (<q25 or >q75)
       • skip the very first bar after a pivot (swing_len==1)
    """
    MOM_Q25: float
    MOM_Q75: float
    def __init__(self, q25: float, q75: float):
        super().__init__()
        self.MOM_Q25, self.MOM_Q75 = q25, q75
        self._swing_len = np.zeros(50, np.int32)
    @export
    def Alg(self, prc: np.ndarray) -> np.ndarray:
        n,t = prc.shape
        pos = np.zeros(n, np.int32)
        for i,s in enumerate(self._st):
            if i in BLACK_LIST: continue
            px = prc[i,-1]
            if s.le is None: s.le = px
            # momentum
            mom = 0.0
            if t >= 6:
                mom = (px - prc[i,-6]) / prc[i,-6]
            # recalibration
            if t >= s.next:
                win = prc[i, t-s.n:t]
                s.thr  = best_threshold(win)
                s.next = t + s.w
                self._swing_len[i] = 1      # new pivot bar starts swing
            else:
                self._swing_len[i] += 1
            if s.thr is None: continue
            # filters
            if self.MOM_Q25 < mom < self.MOM_Q75:     # middle momentum bucket
                continue
            if self._swing_len[i] == 1:               # first bar of swing
                continue
            # causal update & position
            s.dir, s.le = causal_update(px, s.le, s.dir, s.thr)
            pos[i] = s.dir * POSITION_LIMIT
        return pos

# =========================== back-test harness ============================
def run_backtest(trader_cls, prices) -> Dict[str,float]:
    T,n = prices.shape
    trader = trader_cls()
    pos_prev = np.zeros(n, np.int32)
    pnl_vec  = []
    for t in range(1,T):
        pos = trader.Alg(prices[:t+1,:].T)
        pnl = pos_prev * (prices[t]-prices[t-1])         # $ pnl
        comm = COMM_RATE * np.abs(pos-pos_prev) * prices[t]
        pnl_vec.append((pnl-comm).sum())
        pos_prev = pos
    pnl_vec = np.array(pnl_vec)
    return {
        "TotalPnL": pnl_vec.sum(),
        "MeanBar":  pnl_vec.mean(),
        "StdBar":   pnl_vec.std(ddof=0),
        "Sharpe":   pnl_vec.mean()/pnl_vec.std(ddof=0)*np.sqrt(252*6.5*60)
                    if pnl_vec.std(ddof=0)>0 else 0
    }

# ============================= main run ===================================
if __name__ == "__main__":
    prices = np.loadtxt("prices.txt", delimiter=None)  # (T × 50)

    # pre-compute global momentum quartiles on 5-bar returns
    mom_all = (prices[5:] - prices[:-5]) / prices[:-5]
    q25, q75 = np.nanpercentile(mom_all, [25, 75])

    base   = run_backtest(BasePIPTrader, prices)
    filt   = run_backtest(lambda: FilteredPIPTrader(q25, q75), prices)

    print("\n===== Portfolio metrics =====")
    print("{:<10} {:>12} {:>12} {:>12} {:>10}".format(
        "Run", "TotalPnL", "MeanBar", "StdBar", "Sharpe"))
    for tag,res in [("Base",base), ("Filtered",filt)]:
        print(f"{tag:<10} "
              f"{res['TotalPnL']:12,.0f}"
              f"{res['MeanBar']:12,.2f}"
              f"{res['StdBar']:12,.2f}"
              f"{res['Sharpe']:10.2f}")


In [ ]:
#!/usr/bin/env python3
"""
pip_standalone_diagnostics.py
─────────────────────────────
A **single** self-contained script that

1. **Implements** the most-recent PIP trader (“black-list + overflow-safe
   + momentum / swing filters”).
2. **Runs** a full back-test on `prices.txt`.
3. **Prints** headline portfolio stats plus correlation / quintile
   breakdowns for a wide set of indicators so you can spot residual
   weaknesses.

No files are written; everything goes to stdout.
"""
from pathlib import Path
from typing  import Tuple, List, Dict, Optional

import numpy as np
import pandas as pd

# ═════════════════════════ Trader scaffolding ════════════════════════════
def export(fn):  # no-op decorator so back-tester signatures stay intact
    fn._is_exported = True
    return fn


class Trader:    # minimal stub – just to satisfy type expectations
    def Alg(self, prcSoFar: np.ndarray) -> np.ndarray:
        raise NotImplementedError


# ═════════════════════════ Strategy parameters ═══════════════════════════
BLACK_LIST = {4, 5, 8, 22, 23, 28, 48, 49}

BEST_N = [
    200,200,400,200,400,400,100,300,100,200,100,500,100,300,200,100,200,100,200,300,
    100,200,500,100,400,100,100,400,100,100,400,500,300,200,400,500,100,400,100,100,
    100,300,400,500,100,400,500,100,500,300,
]
BEST_W = [
     10, 10,100, 10,100, 10,100, 25, 10,100,100, 50, 25, 10, 50, 25, 10, 25, 50,100,
    100, 10, 10, 50, 25, 25, 10, 50, 25,100, 25, 10,100, 50, 25, 10, 50,100, 50, 25,
     25, 10,100,100, 50, 50, 10, 25, 25, 25,
]

POSITION_LIMIT = 10_000
THR_MIN, THR_MAX, THR_STEP = 0.0, 0.700, 0.0005
THR_GRID = np.arange(THR_MIN, THR_MAX + THR_STEP, THR_STEP)

# ═════════════════════════ Helper functions ══════════════════════════════
def causal_update(px: float, last_extreme: float,
                  direction: int, thr: float) -> Tuple[int, float]:
    """One-bar fully causal PIP update."""
    if direction == 0:
        move = (px - last_extreme) / last_extreme
        if abs(move) >= thr:
            return (1 if move > 0 else -1), px
        return 0, last_extreme
    if direction == 1:
        if px > last_extreme:
            return 1, px
        if (last_extreme - px) / last_extreme >= thr:
            return -1, px
        return 1, last_extreme
    # direction == -1
    if px < last_extreme:
        return -1, px
    if (px - last_extreme) / last_extreme >= thr:
        return 1, px
    return -1, last_extreme


def best_threshold(window: np.ndarray) -> float:
    """Overflow-safe grid search on the trailing window."""
    best_thr, best_pnl = THR_GRID[0], -np.inf
    for thr in THR_GRID:
        d, le = 0, window[0]
        pnl = 0.0
        for p0, p1 in zip(window[:-1], window[1:]):
            d, le = causal_update(p1, le, d, thr)
            pnl += int(d) * POSITION_LIMIT * (p1 - p0)
        if pnl > best_pnl:
            best_pnl, best_thr = pnl, thr
    return best_thr


# ═════════════════════════ Trader implementation ════════════════════════
class CausalPIPTrader(Trader):
    """
    • Skips the eight chronic-loss symbols (BLACK_LIST)
    • Skips the first bar of every new swing
    • Trades only when |5-bar momentum| is in the outer quartiles (<Q25 or >Q75)
    """
    class _S:
        __slots__ = ("n","w","next","thr","dir","le")
        def __init__(self, n:int, w:int):
            self.n, self.w = n, w
            self.next = n
            self.thr  = None
            self.dir  = 0
            self.le   = None

    def __init__(self, mom_q25: float, mom_q75: float):
        super().__init__()
        self.q25, self.q75 = mom_q25, mom_q75
        self._st = [self._S(n,w) for n,w in zip(BEST_N, BEST_W)]
        self._swing_len = np.zeros(50, np.int32)
        print("CausalPIPTrader ready (filters enabled).")

    @export
    def Alg(self, prcSoFar: np.ndarray) -> np.ndarray:
        n,t = prcSoFar.shape
        pos = np.zeros(n, np.int32)
        for i,s in enumerate(self._st):
            if i in BLACK_LIST:      # always flat
                continue
            px = prcSoFar[i,-1]
            # init extreme
            if s.le is None:
                s.le = px
            # momentum
            mom5 = 0.0
            if t >= 6:
                mom5 = (px - prcSoFar[i,-6]) / prcSoFar[i,-6]
            # recalibration
            if t >= s.next:
                win = prcSoFar[i, t-s.n:t]
                s.thr  = best_threshold(win)
                s.next = t + s.w
                self._swing_len[i] = 1
            else:
                self._swing_len[i] += 1
            if s.thr is None:
                continue
            # filters
            if self.q25 < mom5 < self.q75:
                continue
            if self._swing_len[i] == 1:
                continue
            # causal update
            s.dir, s.le = causal_update(px, s.le, s.dir, s.thr)
            pos[i] = s.dir * POSITION_LIMIT
        return pos


# ═════════════════════════ Analysis / diagnostics ═══════════════════════
def run_and_collect(price_file: str = "prices.txt") -> pd.DataFrame:
    prices = np.loadtxt(Path(price_file), delimiter=None)   # (T × 50)
    T, nI  = prices.shape

    # global 5-bar momentum quartiles
    mom5_all = (prices[5:] - prices[:-5]) / prices[:-5]
    q25, q75 = np.nanpercentile(mom5_all, [25, 75])

    trader = CausalPIPTrader(q25, q75)

    rec: Dict[str, List] = {k: [] for k in
        ("inst","bar","price","ret1","vol20","vol40","mom5","mom10",
         "atr20","thr","dir","swing_len","pnl")}

    pos_prev  = np.zeros(nI, np.int32)
    last_dir  = np.zeros(nI, np.int8)
    swing_len = np.zeros(nI, np.int32)
    buf20 = np.full((20, nI), np.nan)
    buf40 = np.full((40, nI), np.nan)
    absdiff = np.abs(np.diff(prices, axis=0))

    for t in range(1, T):
        # indicators
        ret1 = (prices[t] - prices[t-1]) / prices[t-1]
        buf20[t % 20] = ret1
        buf40[t % 40] = ret1
        vol20 = np.nanstd(buf20, axis=0)
        vol40 = np.nanstd(buf40, axis=0)
        mom5  = (prices[t] - prices[t-5]) / prices[t-5] if t >= 5  else np.zeros(nI)
        mom10 = (prices[t]-prices[t-10])/prices[t-10]   if t >= 10 else np.zeros(nI)
        atr20_row = absdiff[max(0,t-19):t+1].mean(axis=0) if t >= 20 else np.zeros(nI)

        # trade
        pos = trader.Alg(prices[:t+1,:].T)
        pnl = pos_prev * (prices[t] - prices[t-1])

        # record
        for i in range(nI):
            rec["inst"].append(i)
            rec["bar"].append(t)
            rec["price"].append(prices[t,i])
            rec["ret1"].append(ret1[i])
            rec["vol20"].append(vol20[i])
            rec["vol40"].append(vol40[i])
            rec["mom5"].append(mom5[i])
            rec["mom10"].append(mom10[i])
            rec["atr20"].append(atr20_row[i])
            st = trader._st[i]
            rec["thr"].append(st.thr)
            rec["dir"].append(st.dir)
            # swing length bookkeeping
            if st.dir == last_dir[i]:
                swing_len[i] += 1
            else:
                swing_len[i] = 1
            last_dir[i] = st.dir
            rec["swing_len"].append(swing_len[i])
            rec["pnl"].append(pnl[i])

        pos_prev = pos.copy()

    return pd.DataFrame(rec)


def corr_and_quintiles(df: pd.DataFrame, metric: str) -> None:
    corr = df[metric].corr(df["pnl"])
    print(f"\n{metric} correlation with PnL : {corr:+.4f}")
    qcol = f"{metric}_q"
    df[qcol] = pd.qcut(df[metric].fillna(0), 5, labels=False, duplicates="drop")
    tbl = (df.groupby(qcol)["pnl"]
             .agg(mean="mean", total="sum", count="count"))
    print(tbl.to_string())


# ════════════════════════════════════════════════════════════════════════
if __name__ == "__main__":
    df = run_and_collect("prices.txt")

    # headline
    total = df["pnl"].sum()
    mean  = df["pnl"].mean()
    std   = df["pnl"].std(ddof=0)
    sharpe= mean/std*np.sqrt(252*6.5*60) if std else 0
    print("=== Portfolio headline ===")
    print(f"Total PnL : {total:,.0f}")
    print(f"Mean/bar  : {mean:,.2f}")
    print(f"Std/bar   : {std:,.2f}")
    print(f"Sharpe    : {sharpe:,.2f}")

    # diagnostics
    for m in ("vol20","vol40","mom5","mom10","atr20","swing_len","thr"):
        corr_and_quintiles(df, m)

    # worst instruments
    worst = (df.groupby("inst")["pnl"].sum()
               .sort_values().head(10))
    print("\n=== Worst 10 instruments ===")
    print(worst.to_string())


In [ ]:
#!/usr/bin/env python3
"""
dynamic_threshold_gridsearch_fast.py  (causal & accelerated)
─────────────────────────────────────────────────────────────
* Per instrument, evaluate every (N, W) in N_GRID × W_GRID:
    – Fit the best pivot-threshold on the last N bars.
    – Trade the **next W bars** with that fixed threshold
      (position = ±POSITION_SIZE shares via causal PIP signal).
    – Re-fit and repeat until the end of the series.
* Compare that dynamic PnL with the best one-shot static threshold.
* Uses Numba JIT + joblib multiprocessing for speed.
"""
from __future__ import annotations
from pathlib   import Path
from typing    import Sequence, Tuple, List

import numpy as np
from numba import njit, prange
from joblib import Parallel, delayed

# ════════════════════════════ CONFIG ═════════════════════════════════════
PRICE_FILE    = "prices-2024.txt"        # whitespace-delimited (T × 50)
POSITION_SIZE = 10_000              # shares long / short
COMM_RATE     = 0.0005              # 5 bp per dollar traded

# Inner threshold grid
THR_MIN   = 0.0
THR_MAX   = 0.700
THR_STEP  = 0.0001
THR_GRID  = np.arange(THR_MIN, THR_MAX + THR_STEP, THR_STEP, dtype=np.float64)

# Outer (N, W) calibration grids
N_GRID: Sequence[int] = [100, 200, 300, 400, 500]
W_GRID: Sequence[int] = [10, 25, 50, 100]

# ═══════════════════════ HELPERS / IO ════════════════════════════════════
def load_prices(fname: str = PRICE_FILE) -> np.ndarray:
    arr = np.loadtxt(Path(fname), delimiter=None)
    if arr.ndim != 2:
        raise ValueError("prices-2024.txt must be 2-D (T × nInst)")
    return arr

# ═══════════════════════ CORE NUMBA JITs ═════════════════════════════════
@njit(cache=True, fastmath=True)
def _causal_dir(series: np.ndarray, thr: float) -> np.ndarray:
    """Return int8 vector {-1,0,+1} for one instrument."""
    n   = series.size
    out = np.zeros(n, np.int8)
    le  = series[0]      # last extreme
    d   = 0              # direction 0/±1

    for i in range(1, n):
        move = (series[i] - le) / le
        if d == 0:
            if abs(move) >= thr:
                d  = 1 if move > 0 else -1
                le = series[i]
        elif d == 1:
            if series[i] > le:
                le = series[i]
            elif (le - series[i]) / le >= thr:
                d  = -1
                le = series[i]
        else:            # d == -1
            if series[i] < le:
                le = series[i]
            elif (series[i] - le) / le >= thr:
                d  = 1
                le = series[i]
        out[i] = d
    return out


@njit(cache=True, fastmath=True)
def _pnl_static(series: np.ndarray,
                thr: float,
                pos_size: int,
                comm_rate: float) -> float:
    dir_vec = _causal_dir(series, thr)
    pos     = dir_vec.astype(np.int32) * pos_size

    pnl = 0.0
    comm = 0.0
    for i in range(1, series.size):
        pnl  += pos[i-1] * (series[i] - series[i-1])
        comm += comm_rate * abs(pos[i] - pos[i-1]) * series[i]
    return pnl - comm


@njit(cache=True, fastmath=True)
def _best_thr(window: np.ndarray,
              thr_grid: np.ndarray,
              pos_size: int,
              comm_rate: float) -> float:
    """Return threshold with max net-PnL on *window*."""
    best_thr = thr_grid[0]
    best_pnl = -1e30
    for thr in thr_grid:
        pnl = _pnl_static(window, thr, pos_size, comm_rate)
        if pnl > best_pnl:
            best_pnl = pnl
            best_thr = thr
    return best_thr


@njit(cache=True, fastmath=True)
def _pnl_dynamic(series: np.ndarray,
                 N: int, W: int,
                 thr_grid: np.ndarray,
                 pos_size: int,
                 comm_rate: float) -> float:
    """
    True walk-forward: fit on [t-N, t-1], trade bars t … t+W-1,
    then slide window W bars forward.
    """
    T   = series.size
    pos = np.zeros(T, np.int32)

    if N >= T:
        return 0.0

    seg_start = N
    while seg_start < T:
        # fit on trailing N bars *ending at seg_start-1*
        thr = _best_thr(series[seg_start-N: seg_start],
                        thr_grid, pos_size, comm_rate)

        seg_end = min(seg_start + W, T)
        # trade this segment with fixed thr
        d_vec = _causal_dir(series[:seg_end], thr)
        for t in range(seg_start, seg_end):
            pos[t] = d_vec[t] * pos_size

        seg_start += W

    # ------------ compute net PnL -------------------------------------
    pnl = comm = 0.0
    for i in range(1, T):
        pnl  += pos[i-1] * (series[i] - series[i-1])
        comm += comm_rate * abs(pos[i] - pos[i-1]) * series[i]
    return pnl - comm

# ═════════════ wrappers for multiprocessing / evaluation ════════════════
def eval_instrument(idx: int,
                    series: np.ndarray
                    ) -> Tuple[int,int,float,float,float]:
    """Return (best_N, best_W, dyn_PnL, static_thr, static_PnL)."""
    # static best threshold
    best_thr, best_static = -1.0, -1e30
    for thr in THR_GRID:
        pnl = _pnl_static(series, thr, POSITION_SIZE, COMM_RATE)
        if pnl > best_static:
            best_thr, best_static = thr, pnl

    # dynamic grid
    best_N = best_W = 0
    best_dyn = -1e30
    for N in N_GRID:
        if N >= series.size:
            continue
        for W in W_GRID:
            pnl = _pnl_dynamic(series, N, W, THR_GRID,
                               POSITION_SIZE, COMM_RATE)
            if pnl > best_dyn:
                best_dyn, best_N, best_W = pnl, N, W

    return best_N, best_W, best_dyn, best_thr, best_static


# ═══════════════════════════ MAIN ════════════════════════════════════════
def main() -> None:
    prices = load_prices()
    T, nInst = prices.shape
    print(f"Loaded price matrix: {T:,d} bars × {nInst} instruments\n")

    # parallel across CPU cores
    results: List[Tuple[int,int,float,float,float]] = Parallel(
        n_jobs=-1, backend="loky", prefer="processes")(
        delayed(eval_instrument)(i, prices[:, i]) for i in range(nInst))

    hdr = ("Inst", "Best-N", "Best-W", "Dyn-PnL", "StaticThr", "Static-PnL")
    print(f"{hdr[0]:>5}  {hdr[1]:>7}  {hdr[2]:>7}  {hdr[3]:>12}  "
          f"{hdr[4]:>10}  {hdr[5]:>12}")
    print("─"*72)

    tot_dyn = tot_static = 0.0
    for i, (N_opt, W_opt, pnl_dyn, thr_stat, pnl_stat) in enumerate(results):
        tot_dyn    += pnl_dyn
        tot_static += pnl_stat
        print(f"{i:5d}  {N_opt:7d}  {W_opt:7d}  {pnl_dyn:12,.2f}  "
              f"{thr_stat:10.4f}  {pnl_stat:12,.2f}")

    print("─"*72)
    print(f"Σ Net PnL (dynamic): {tot_dyn:,.2f}")
    print(f"Σ Net PnL (static) : {tot_static:,.2f}")


if __name__ == "__main__":
    main()


In [ ]:
#!/usr/bin/env python3
"""
pip_trader_with_plot.py
───────────────────────
Standalone script that:

1. Implements the dynamic PIP trader exactly as before.
2. Loads `prices.txt`, runs the trader on 50 instruments.
3. Plots each series with green shading for long and red for short.
"""
from pathlib import Path
from typing    import List, Optional, Tuple

import numpy as np
import matplotlib.pyplot as plt


# ──────────────── Trader scaffolding (standalone) ──────────────────────
def export(fn):
    """No-op decorator to mark exported methods."""
    fn._is_exported = True
    return fn

class Trader:
    """Minimal base class for our model."""
    def Alg(self, prcSoFar: np.ndarray) -> np.ndarray:
        raise NotImplementedError


# ─────────────────── strategy hyper‐parameters ─────────────────────────
BEST_N: List[int] = [
    200, 200, 400, 200, 400, 400, 100, 300, 100, 200,
    100, 500, 100, 300, 200, 100, 200, 100, 200, 300,
    100, 200, 500, 100, 400, 100, 100, 400, 100, 100,
    400, 500, 300, 200, 400, 500, 100, 400, 100, 100,
    100, 300, 400, 500, 100, 400, 500, 100, 500, 300,
]
BEST_W: List[int] = [
     10,  10, 100,  10, 100,  10, 100,  25,  10, 100,
    100,  50,  25,  10,  50,  25,  10,  25,  50, 100,
    100,  10,  10,  50,  25,  25,  10,  50,  25, 100,
     25,  10, 100,  50,  25,  10,  50, 100,  50,  25,
     25,  10, 100, 100,  50,  50,  10,  25,  25,  25,
]

POSITION_LIMIT   = 10_000
THR_MIN, THR_MAX, THR_STEP = 0.0, 0.700, 0.0005
THR_GRID         = np.arange(THR_MIN, THR_MAX + THR_STEP, THR_STEP, dtype=float)


# ───────────────────────── helper functions ────────────────────────────
def causal_update(px: float,
                  last_extreme: float,
                  direction: Optional[int],
                  thr: float) -> Tuple[int, float]:
    """
    One‐step causal PIP update.
    Returns (new_direction, new_last_extreme).
    """
    if direction == 0 or direction is None:
        move = (px - last_extreme) / last_extreme
        if abs(move) >= thr:
            return (1 if move > 0 else -1), px
        return 0, last_extreme

    if direction == 1:  # up‐swing
        if px > last_extreme:
            return 1, px
        if (last_extreme - px) / last_extreme >= thr:
            return -1, px
        return 1, last_extreme

    # direction == -1 (down‐swing)
    if px < last_extreme:
        return -1, px
    if (px - last_extreme) / last_extreme >= thr:
        return 1, px
    return -1, last_extreme


def best_threshold(window: np.ndarray) -> float:
    """
    Overflow‐safe grid‐search of pivot threshold on `window`.
    """
    best_thr, best_pnl = THR_GRID[0], -np.inf
    for thr in THR_GRID:
        dir_vec = np.zeros(len(window), dtype=np.int8)
        le, d = window[0], 0
        for k in range(1, len(window)):
            d, le = causal_update(window[k], le, d, thr)
            dir_vec[k] = d
        pnl = np.sum(dir_vec[:-1].astype(np.int32) *
                     POSITION_LIMIT *
                     np.diff(window))
        if pnl > best_pnl:
            best_pnl, best_thr = pnl, thr
    return best_thr


# ────────────────────────── trader class ──────────────────────────────
class CausalPIPTrader(Trader):
    """
    Per‐instrument rolling‐window PIP trader.
    """

    class _InstrState:
        __slots__ = ("n", "w", "next_calib", "thr",
                     "dir", "last_extreme")

        def __init__(self, n: int, w: int):
            self.n = n
            self.w = w
            self.next_calib = n      # first calibration at bar N
            self.thr = None          # threshold once fit
            self.dir = 0            # current direction
            self.last_extreme = None

    def __init__(self):
        super().__init__()
        if len(BEST_N) != 50 or len(BEST_W) != 50:
            raise ValueError("Need exactly 50 (N, W) pairs.")
        self._states = [self._InstrState(n, w)
                        for n, w in zip(BEST_N, BEST_W)]
        print("CausalPIPTrader initialized.")

    @export
    def Alg(self, prcSoFar: np.ndarray) -> np.ndarray:
        nInst, t_seen = prcSoFar.shape
        if nInst != 50:
            raise ValueError("Expecting 50 instruments.")
        positions = np.zeros(nInst, dtype=np.int32)

        for i in range(nInst):
            st = self._states[i]
            price = prcSoFar[i, -1]

            # initialize last extreme
            if st.last_extreme is None:
                st.last_extreme = price

            # recalibrate threshold when due
            if t_seen >= st.next_calib:
                window = prcSoFar[i, t_seen - st.n : t_seen]
                st.thr = best_threshold(window)
                st.next_calib += st.w
                print(f"[t={t_seen:4d}] Inst {i:2d} thr={st.thr:.4f}")

            # skip until first calibration
            if st.thr is None:
                continue

            # one‐step causal update
            st.dir, st.last_extreme = causal_update(
                price, st.last_extreme, st.dir, st.thr
            )
            positions[i] = st.dir * POSITION_LIMIT

        return positions


# ─────────────────────── simulation & plotting ──────────────────────
def run_and_plot(prices_path: str = "prices.txt") -> None:
    # load price matrix (T × 50)
    prices = np.loadtxt(Path(prices_path), delimiter=None)
    T, nInst = prices.shape

    trader = CausalPIPTrader()

    # record positions at each bar
    positions = np.zeros((T, nInst), dtype=int)
    for t in range(1, T + 1):
        pos = trader.Alg(prices[:t, :].T)
        positions[t-1] = pos

    # plot each instrument
    fig, axes = plt.subplots(5, 10, figsize=(20, 10), sharex=True, sharey=True)
    axes = axes.flatten()

    for i, ax in enumerate(axes):
        series = prices[:, i]
        pos    = positions[:, i]
        times  = np.arange(T)

        ax.plot(times, series, color="black", linewidth=1)

        ax.fill_between(times, series.min(), series,
                        where=pos > 0,
                        facecolor="green", alpha=0.3,
                        step="post")

        ax.fill_between(times, series.max(), series,
                        where=pos < 0,
                        facecolor="red", alpha=0.3,
                        step="post")

        ax.set_title(f"Inst {i:02d}", fontsize=8)
        ax.set_xticks([])
        ax.set_yticks([])

    fig.suptitle("Dynamic PIP Trader: Long (green) / Short (red)", fontsize=16)
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.show()


if __name__ == "__main__":
    run_and_plot()


In [ ]:
#!/usr/bin/env python3
"""
pip_trader_with_plot_individual.py
────────────────────────
Standalone script that:

1. Implements the dynamic PIP trader exactly as before.
2. Loads `prices.txt`, runs the trader on 50 instruments.
3. Plots each series in individual figures with green shading for long and red for short.
"""
from pathlib import Path
from typing    import List, Optional, Tuple

import numpy as np
import matplotlib.pyplot as plt


# ──────────────────── Trader scaffolding ─────────────────────
def export(fn):
    """No-op decorator to mark exported methods."""
    fn._is_exported = True
    return fn

class Trader:
    """Minimal base class for our model."""
    def Alg(self, prcSoFar: np.ndarray) -> np.ndarray:
        raise NotImplementedError


# ────────────────────── strategy hyper‐parameters ───────────────────
BEST_N: List[int] = [
    200, 200, 400, 200, 400, 400, 100, 300, 100, 200,
    100, 500, 100, 300, 200, 100, 200, 100, 200, 300,
    100, 200, 500, 100, 400, 100, 100, 400, 100, 100,
    400, 500, 300, 200, 400, 500, 100, 400, 100, 100,
    100, 300, 400, 500, 100, 400, 500, 100, 500, 300,
]
BEST_W: List[int] = [
     10,  10, 100,  10, 100,  10, 100,  25,  10, 100,
    100,  50,  25,  10,  50,  25,  10,  25,  50, 100,
    100,  10,  10,  50,  25,  25,  10,  50,  25, 100,
     25,  10, 100,  50,  25,  10,  50, 100,  50,  25,
     25,  10, 100, 100,  50,  50,  10,  25,  25,  25,
]

POSITION_LIMIT   = 10_000
THR_MIN, THR_MAX, THR_STEP = 0.0, 0.700, 0.0005
THR_GRID         = np.arange(THR_MIN, THR_MAX + THR_STEP, THR_STEP, dtype=float)


# ────────────────────── helper functions ─────────────────────────
def causal_update(px: float,
                  last_extreme: float,
                  direction: Optional[int],
                  thr: float) -> Tuple[int, float]:
    """
    One‐step causal PIP update.
    Returns (new_direction, new_last_extreme).
    """
    if direction == 0 or direction is None:
        move = (px - last_extreme) / last_extreme
        if abs(move) >= thr:
            return (1 if move > 0 else -1), px
        return 0, last_extreme
    if direction == 1:  # up‐swing
        if px > last_extreme:
            return 1, px
        if (last_extreme - px) / last_extreme >= thr:
            return -1, px
        return 1, last_extreme
    # direction == -1 (down‐swing)
    if px < last_extreme:
        return -1, px
    if (px - last_extreme) / last_extreme >= thr:
        return 1, px
    return -1, last_extreme


def best_threshold(window: np.ndarray) -> float:
    """
    Overflow‐safe grid‐search of pivot threshold on `window`.
    """
    best_thr, best_pnl = THR_GRID[0], -np.inf
    for thr in THR_GRID:
        dir_vec = np.zeros(len(window), dtype=np.int8)
        le, d = window[0], 0
        for k in range(1, len(window)):
            d, le = causal_update(window[k], le, d, thr)
            dir_vec[k] = d
        pnl = np.sum(dir_vec[:-1].astype(np.int32) *
                     POSITION_LIMIT *
                     np.diff(window))
        if pnl > best_pnl:
            best_pnl, best_thr = pnl, thr
    return best_thr


# ───────────────────────── trader class ───────────────────────────
class CausalPIPTrader(Trader):
    """
    Per‐instrument rolling‐window PIP trader.
    """

    class _InstrState:
        __slots__ = ("n", "w", "next_calib", "thr",
                     "dir", "last_extreme")

        def __init__(self, n: int, w: int):
            self.n = n
            self.w = w
            self.next_calib = n      # first calibration at bar N
            self.thr = None          # threshold once fit
            self.dir = 0            # current direction
            self.last_extreme = None

    def __init__(self):
        super().__init__()
        if len(BEST_N) != 50 or len(BEST_W) != 50:
            raise ValueError("Need exactly 50 (N, W) pairs.")
        self._states = [self._InstrState(n, w)
                        for n, w in zip(BEST_N, BEST_W)]
        print("CausalPIPTrader initialized.")

    @export
    def Alg(self, prcSoFar: np.ndarray) -> np.ndarray:
        nInst, t_seen = prcSoFar.shape
        if nInst != 50:
            raise ValueError("Expecting 50 instruments.")
        positions = np.zeros(nInst, dtype=np.int32)

        for i in range(nInst):
            st = self._states[i]
            price = prcSoFar[i, -1]

            # initialize last extreme
            if st.last_extreme is None:
                st.last_extreme = price

            # recalibrate threshold when due
            if t_seen >= st.next_calib:
                window = prcSoFar[i, t_seen - st.n : t_seen]
                st.thr = best_threshold(window)
                st.next_calib += st.w
                print(f"[t={t_seen:4d}] Inst {i:2d} thr={st.thr:.4f}")

            # skip until first calibration
            if st.thr is None:
                continue

            # one‐step causal update
            st.dir, st.last_extreme = causal_update(
                price, st.last_extreme, st.dir, st.thr
            )
            positions[i] = st.dir * POSITION_LIMIT

        return positions


# ──────────────────── run and cache outputs ───────────────────────
prices = np.loadtxt(Path("prices.txt"), delimiter=None)  # shape (T,50)
T, nInst = prices.shape
trader = CausalPIPTrader()
_cached_positions = np.zeros((T, nInst), dtype=int)

for t in range(1, T + 1):
    _cached_positions[t-1] = trader.Alg(prices[:t, :].T)

print("Finished running trader and caching positions.")


In [ ]:
# Cell 2: Plot each instrument separately

import numpy as np
import matplotlib.pyplot as plt

# Assume `prices` and `_cached_positions` are already in namespace from Cell 1

T, nInst = prices.shape

for i in range(nInst):
    fig, ax = plt.subplots(figsize=(10, 4))
    series = prices[:, i]
    pos    = _cached_positions[:, i]
    times  = np.arange(T)

    ax.plot(times, series, color="black", linewidth=1)
    ax.fill_between(times, series.min(), series,
                    where=pos > 0, facecolor="green", alpha=0.3, step="mid")
    ax.fill_between(times, series.max(), series,
                    where=pos < 0, facecolor="red", alpha=0.3, step="mid")

    ax.set_title(f"Instrument {i:02d}")
    ax.set_xlabel("Timestep")
    ax.set_ylabel("Price")
    plt.tight_layout()
    plt.show()


In [ ]:
# Cell 2: Plot each instrument separately with cumulative PnL and commission

import numpy as np
import matplotlib.pyplot as plt

# Commission rate: 0.05% per dollar traded
commission_rate = 0.0005

# Assume `prices` (T × nInst) and `_cached_positions` (T × nInst) are already in namespace
T, nInst = prices.shape
times = np.arange(T)

for i in range(nInst):
    series = prices[:, i]
    pos    = _cached_positions[:, i]

    # ——— Compute PnL and Commission —————————————
    # per-bar PnL: position at t * (price[t+1] - price[t])
    price_diff    = np.diff(series)
    trade_pnl     = pos[:-1] * price_diff

    # commission on each trade: abs(change in position) * price when trade occurs
    # assume trade executes at the bar's end price series[1:]
    pos_change    = np.abs(pos[1:] - pos[:-1])
    commission    = commission_rate * pos_change * series[1:]

    # net PnL after commission
    net_pnl       = trade_pnl - commission

    # total and cumulative PnL
    total_pnl     = net_pnl.sum()
    cum_pnl       = np.concatenate(([0], np.cumsum(net_pnl)))

    # ——— Price Chart with Shading —————————————
    fig, ax = plt.subplots(figsize=(10, 4))
    ax.plot(times, series, color="black", linewidth=1, label="Price")
    ax.fill_between(times, series.min(), series,
                    where=pos > 0, facecolor="green", alpha=0.3, step="mid",
                    label="Long")
    ax.fill_between(times, series.max(), series,
                    where=pos < 0, facecolor="red", alpha=0.3, step="mid",
                    label="Short")

    ax.set_title(f"Instrument {i:02d} — Total Net PnL: {total_pnl:.2f}")
    ax.set_xlabel("Timestep")
    ax.set_ylabel("Price")
    ax.legend(loc="upper left", fontsize="small")
    plt.tight_layout()
    plt.show()

    # ——— Cumulative PnL Plot —————————————
    fig, ax2 = plt.subplots(figsize=(10, 2))
    ax2.plot(times, cum_pnl, color="blue", linewidth=1)
    ax2.set_title(f"Instrument {i:02d} — Cumulative Net PnL")
    ax2.set_xlabel("Timestep")
    ax2.set_ylabel("Cumulative PnL")
    plt.tight_layout()
    plt.show()


In [ ]:
#!/usr/bin/env python3
"""
dynamic_pip_trader.py
─────────────────────
Live-trading implementation of the **window-N / step-W** re-calibration
logic with an entry buffer: stays neutral for the first `ENTRY_DELAY` bars
of any new swing before taking a position.

Strategy recap
--------------
│  • For instrument *i* we fix (N[i], W[i]) found in the prior grid-search.
│  • Once we have at least N[i] bars of history:
│        – Find the best pivot-threshold on the last N[i] bars
│          (grid 0 → 0.700 step 0.0005).
│        – Trade the NEXT W[i] bars with that fixed threshold.
│        – Repeat.
│  • Upon any new swing (direction change), wait `ENTRY_DELAY` bars
│    of that swing before entering.
│  • Position = ±POSITION_LIMIT shares using the causal PIP signal.
│  • Commission/slippage handled by the back-tester / broker layer.
"""
from pathlib import Path
from typing  import List, Optional, Tuple

import numpy as np

# ─────────────────── hyper-parameters from the grid-search ──────────────
BEST_N: List[int] = [
    200, 200, 400, 200, 400, 400, 100, 300, 100, 200,
    100, 500, 100, 300, 200, 100, 200, 100, 200, 300,
    100, 200, 500, 100, 400, 100, 100, 400, 100, 100,
    400, 500, 300, 200, 400, 500, 100, 400, 100, 100,
    100, 300, 400, 500, 100, 400, 500, 100, 500, 300,
]
BEST_W: List[int] = [
     10,  10, 100,  10, 100,  10, 100,  25,  10, 100,
    100,  50,  25,  10,  50,  25,  10,  25,  50, 100,
    100,  10,  10,  50,  25,  25,  10,  50,  25, 100,
     25,  10, 100,  50,  25,  10,  50, 100,  50,  25,
     25,  10, 100, 100,  50,  50,  10,  25,  25,  25,
]

POSITION_LIMIT = 10_000
THR_MIN, THR_MAX, THR_STEP = 0.0, 0.700, 0.0005
THR_GRID = np.arange(THR_MIN, THR_MAX + THR_STEP, THR_STEP, dtype=float)
ENTRY_DELAY = 2   # bars to wait after swing change before trading


# ────────────────────────── helper functions ────────────────────────────
def causal_update(px: float,
                  last_extreme: float,
                  direction: Optional[int],
                  thr: float) -> Tuple[int, float]:
    """
    O(1) causal PIP update for a single new price.
    Returns (new_direction, new_last_extreme).
    """
    if direction == 0 or direction is None:
        move = (px - last_extreme) / last_extreme
        if abs(move) >= thr:
            return (1 if move > 0 else -1), px
        return 0, last_extreme

    if direction == 1:  # up-swing
        if px > last_extreme:
            return 1, px
        if (last_extreme - px) / last_extreme >= thr:
            return -1, px
        return 1, last_extreme

    # direction == -1 (down-swing)
    if px < last_extreme:
        return -1, px
    if (px - last_extreme) / last_extreme >= thr:
        return 1, px
    return -1, last_extreme


def best_threshold(window: np.ndarray) -> float:
    """
    Grid search for the threshold that maximises net PnL on `window`.
    Uses causal-PnL definition.
    """
    best_thr = THR_GRID[0]
    best_pnl = -np.inf
    for thr in THR_GRID:
        dir_vec = np.zeros_like(window, dtype=np.int8)
        last_extreme = window[0]
        d = 0
        for i in range(1, len(window)):
            d, last_extreme = causal_update(window[i], last_extreme, d, thr)
            dir_vec[i] = d
        pos = dir_vec.astype(np.int32) * POSITION_LIMIT
        pnl = np.sum(pos[:-1] * np.diff(window))
        if pnl > best_pnl:
            best_pnl, best_thr = pnl, thr
    return best_thr


# ───────────────────────────── trader class ──────────────────────────────
class CausalPIPTrader(Trader):
    """
    Rolling-window causal-PIP trader with entry delay buffer.
    """

    class _InstrState:
        __slots__ = ("n", "w", "next_calib", "thr",
                     "dir", "last_extreme", "swing_len")

        def __init__(self, n: int, w: int):
            self.n = n
            self.w = w
            self.next_calib = n
            self.thr = None
            self.dir = 0
            self.last_extreme = None
            self.swing_len = 0

    def __init__(self):
        super().__init__()
        if len(BEST_N) != 50 or len(BEST_W) != 50:
            raise ValueError("Need 50 (N,W) pairs.")
        self._states = [self._InstrState(N, W)
                        for N, W in zip(BEST_N, BEST_W)]
        print("DynamicPIPTrader ready with entry delay.")

    @export
    def Alg(self, prcSoFar: np.ndarray) -> np.ndarray:
        nInst, t_seen = prcSoFar.shape
        if nInst != 50:
            raise ValueError("Expecting 50 instruments.")

        positions = np.zeros(nInst, dtype=np.int32)

        for i in range(nInst):
            st = self._states[i]
            price = prcSoFar[i, -1]

            if st.last_extreme is None:
                st.last_extreme = price

            # recalibrate threshold when due
            if t_seen >= st.next_calib:
                window = prcSoFar[i, t_seen - st.n: t_seen]
                st.thr = best_threshold(window)
                st.next_calib += st.w
                # reset swing length on recal
                st.swing_len = 0
                print(f"[t={t_seen:4d}] Inst {i:2d} thr={st.thr:.4f}")

            if st.thr is None:
                continue

            # causal update
            new_dir, new_le = causal_update(price, st.last_extreme, st.dir, st.thr)
            # detect swing change
            if new_dir != st.dir:
                st.swing_len = 1
            else:
                st.swing_len += 1
            st.dir = new_dir
            st.last_extreme = new_le

            # entry delay: only trade if swing_len > ENTRY_DELAY
            if st.swing_len > ENTRY_DELAY and st.dir != 0:
                positions[i] = st.dir * POSITION_LIMIT
            else:
                positions[i] = 0

        return positions

__all__ = ["CausalPIPTrader"]


In [ ]:
# Cell – run this one cell in a fresh notebook
# ===========================================
import numpy as np
from pathlib import Path
from typing import List, Optional, Tuple, Callable, Dict

# -------------  STATIC GRIDS & CONSTANTS  -----------------
BEST_N = [
    200, 200, 400, 200, 400, 400, 100, 300, 100, 200,
    100, 500, 100, 300, 200, 100, 200, 100, 200, 300,
    100, 200, 500, 100, 400, 100, 100, 400, 100, 100,
    400, 500, 300, 200, 400, 500, 100, 400, 100, 100,
    100, 300, 400, 500, 100, 400, 500, 100, 500, 300,
]

BEST_W = [
     10,  10, 100,  10, 100,  10, 100,  25,  10, 100,
    100,  50,  25,  10,  50,  25,  10,  25,  50, 100,
    100,  10,  10,  50,  25,  25,  10,  50,  25, 100,
     25,  10, 100,  50,  25,  10,  50, 100,  50,  25,
     25,  10, 100, 100,  50,  50,  10,  25,  25,  25,
]

THR_MIN, THR_MAX, THR_STEP = 0.0, 0.700, 0.0005
THR_GRID  = np.arange(THR_MIN, THR_MAX + THR_STEP, THR_STEP)
POSITION_LIMIT = 10_000
BLACK_LIST = {4, 5, 8, 22, 23, 28, 48, 49}
ENTRY_DELAY = 2                 # bars to wait after a fresh pivot
# -----------------------------------------------------------

# ---------  LOW-LEVEL UTILITIES (unchanged)  ---------------
def causal_update(px: float, le: float, d: int, thr: float) -> Tuple[int,float]:
    if d == 0:
        mv=(px-le)/le
        if abs(mv)>=thr: return (1 if mv>0 else -1), px
        return 0, le
    if d == 1:
        if px>le: return 1,px
        if (le-px)/le>=thr: return -1,px
        return 1,le
    if px<le: return -1,px
    if (px-le)/le>=thr: return 1,px
    return -1,le

def best_threshold(win: np.ndarray) -> float:
    best_thr, best_pnl = THR_GRID[0], -np.inf
    for thr in THR_GRID:
        d, le, pnl = 0, win[0], 0.0
        for p0, p1 in zip(win[:-1], win[1:]):
            d, le = causal_update(p1, le, d, thr)
            pnl  += d * POSITION_LIMIT * (p1 - p0)
        if pnl > best_pnl:
            best_pnl, best_thr = pnl, thr
    return best_thr
# -----------------------------------------------------------

# ---------------------  BASE TRADER  -----------------------
class BasePIPTrader:
    class _S: __slots__="n","w","next","thr","dir","le","delay"
    def __init__(self, pos_mult:Callable[[int,int],int]|None=None):
        self._st=[]
        for n,w in zip(BEST_N,BEST_W):
            s=self._S(); s.n,s.w,s.next,s.thr,s.dir,s.le,s.delay=n,w,n,None,0,None,0
            self._st.append(s)
        self._pos_mult = pos_mult or (lambda i,t: POSITION_LIMIT)
    def Alg(self, prices_t:np.ndarray, t:int)->np.ndarray:
        """prices_t shape = (nInst, t+1)"""
        n = prices_t.shape[0]
        pos = np.zeros(n, np.int32)
        for i,s in enumerate(self._st):
            px = prices_t[i,-1]; mult=self._pos_mult(i,t)
            if s.le is None: s.le = px
            if t >= s.next:
                s.thr  = best_threshold(prices_t[i, t-s.n:t])
                s.next = t + s.w
                s.delay = ENTRY_DELAY          # entry-delay variant
            if s.thr is None or s.delay>0:
                s.delay = max(0, s.delay-1)
                continue
            s.dir, s.le = causal_update(px, s.le, s.dir, s.thr)
            pos[i] = s.dir * mult
        return pos
# -----------------------------------------------------------

# -------- SINGLE-IDEA VARIANTS  ----------------------------
def mom_q_filter(mom_all):
    q25,q75 = np.nanpercentile(mom_all,[25,75])
    return lambda mom: (mom<q25) or (mom>q75)

def vol_q_filter(vol_all):
    med = np.nanmedian(vol_all)
    return lambda v: v>=med

class MomFilteredTrader(BasePIPTrader):
    def __init__(self, P):
        self._mom_allowed = mom_q_filter((P[:,5:]-P[:,:-5])/P[:,:-5])
        super().__init__()
    def Alg(self,P,t):
        pos=super().Alg(P,t)
        if t>=6:
            mom=(P[:,t]-P[:,t-5])/P[:,t-5]
            pos[~np.vectorize(self._mom_allowed)(mom)] = 0
        return pos

class VolFilteredTrader(BasePIPTrader):
    def __init__(self,P):
        self._vol_allowed = vol_q_filter(np.abs(P[:,1:]-P[:,:-1]))
        super().__init__()
    def Alg(self,P,t):
        pos=super().Alg(P,t)
        if t>=20:
            vol=np.abs(P[:,t]-P[:,t-1])
            pos[~np.vectorize(self._vol_allowed)(vol)] = 0
        return pos

class BlackListTrader(BasePIPTrader):
    def Alg(self,P,t):
        pos=super().Alg(P,t); pos[list(BLACK_LIST)]=0; return pos

class AdaptiveSizeTrader(BasePIPTrader):
    def __init__(self,P):
        atr20=np.zeros_like(P); atr20[:,20:]=np.cumsum(np.abs(P[:,1:]-P[:,:-1]),axis=1)[:,19:]/20
        super().__init__(lambda i,t:int(min(2*POSITION_LIMIT,
                               POSITION_LIMIT*0.5*P[i,t]/max(1e-6,atr20[i,t]))))
# -----------------------------------------------------------

# ---------------------  FAST BACK-TEST  --------------------
def backtest(P:np.ndarray, trader_cls)->Dict[str,float]:
    """P is (T, nInst)  ; we transpose once for speed."""
    P = P.T                    # → shape (50, T)
    n,T = P.shape
    tr  = trader_cls(P) if trader_cls not in (BasePIPTrader,) else trader_cls()
    pos_prev = np.zeros(n, np.int32); pnl=[]
    for t in range(1,T):
        pos = tr.Alg(P[:,:t+1], t)
        pnl.append((pos_prev*(P[:,t]-P[:,t-1])).sum())
        pos_prev = pos
    pnl=np.array(pnl)
    return {"PnL": pnl.sum(), "Sharpe": pnl.mean()/pnl.std(ddof=0)*np.sqrt(252*6.5*60)}

# ---------------------------  RUN  -------------------------
prices = np.loadtxt("prices.txt")          # (T × 50)

traders=[("Base",BasePIPTrader),
         ("EntryDelay",BasePIPTrader),     # already has delay
         ("Momentum",MomFilteredTrader),
         ("VolFilter",VolFilteredTrader),
         ("BlackList",BlackListTrader),
         ("AdaptiveSize",AdaptiveSizeTrader)]

print(f"{'Model':<12}{'TotalPnL':>12}{'Sharpe':>9}")
print("-"*33)
for name,cls in traders:
    res = backtest(prices, cls)
    print(f"{name:<12}{res['PnL']:12,.0f}{res['Sharpe']:9.2f}")


In [ ]:
#!/usr/bin/env python3
"""
per_instrument_gridsearch.py

Grid-search N and W **per instrument** for the dynamic PIP trader,
accounting for commission on dollar turnover.  Prints progress and
a final summary table with best (N, W), total PnL and PnL std for
each of the 50 instruments.
"""

import numpy as np
import pandas as pd
from itertools import product
from pathlib import Path

# ────────────────────────── hyper-parameters ──────────────────────────── #
N_VALUES         = [100, 200, 300, 400]
W_VALUES         = [50, 100, 200]
COMMISSION_RATE  = 0.001     # 0.05% per dollar traded
POSITION_LIMIT   = 10_000.0
THR_GRID         = np.arange(0.0, 0.700+0.0005, 0.0005)

# ───────────────────────── helper functions ────────────────────────────── #
def causal_update(px: float, le: float, d: int, thr: float):
    if d == 0:
        mv = (px - le) / le
        if abs(mv) >= thr:
            return (1 if mv > 0 else -1), px
        return 0, le
    if d == 1:
        if px > le:
            return 1, px
        if (le - px) / le >= thr:
            return -1, px
        return 1, le
    # d == -1
    if px < le:
        return -1, px
    if (px - le) / le >= thr:
        return 1, px
    return -1, le

def best_threshold(window: np.ndarray, thr_grid: np.ndarray, position_limit: float) -> float:
    """
    Find threshold that maximizes PnL over `window` using
    the causal_update logic and given position_limit.
    """
    best_thr = thr_grid[0]
    best_pnl = -np.inf
    for thr in thr_grid:
        d, le = 0, window[0]
        pnl = 0.0
        for p0, p1 in zip(window[:-1], window[1:]):
            d, le = causal_update(p1, le, d, thr)
            pnl += d * position_limit * (p1 - p0)
        if pnl > best_pnl:
            best_pnl, best_thr = pnl, thr
    return best_thr

def backtest_instrument(series: np.ndarray,
                        N: int,
                        W: int,
                        commission_rate: float,
                        thr_grid: np.ndarray,
                        position_limit: float) -> np.ndarray:
    """
    Back-test one instrument price `series` (length T).
    Re-calibrate threshold every W bars using last Nbars.
    Returns net-PnL time series of length T-1 (commission deducted).
    """
    T = series.size
    pnl = np.zeros(T-1, dtype=float)
    # state
    next_cal = N
    thr      = None
    d        = 0
    le       = series[0]
    prev_pos = 0.0

    for t in range(1, T):
        px = series[t]
        # recalibrate threshold
        if t >= next_cal:
            window = series[max(0, t-N): t]
            thr = best_threshold(window, thr_grid, position_limit)
            next_cal += W

        # decide position
        if thr is not None:
            d, le = causal_update(px, le, d, thr)
        pos = d * position_limit

        # compute bar PnL
        price_diff = px - series[t-1]
        trade_pnl  = prev_pos * price_diff

        # commission on dollar turnover
        turnover   = abs(pos - prev_pos) * px
        commission = commission_rate * turnover

        pnl[t-1]   = trade_pnl - commission
        prev_pos   = pos

    return pnl

# ──────────────────────── main grid-search ───────────────────────────── #
if __name__ == "__main__":
    # load price data: shape (T,50) → transpose to (50,T)
    prices = np.loadtxt(Path("prices.txt"), delimiter=None).T

    results = []
    for inst in range(prices.shape[0]):
        series = prices[inst]
        best_pnl = -np.inf
        best_std = np.nan
        best_N   = None
        best_W   = None

        print(f"\n--- Instrument {inst:02d} ---")
        for N, W in product(N_VALUES, W_VALUES):
            print(f" Testing N={N:3d}, W={W:3d} ...", end="", flush=True)
            pnl_series = backtest_instrument(
                series,
                N,
                W,
                COMMISSION_RATE,
                THR_GRID,
                POSITION_LIMIT
            )
            total_pnl = pnl_series.sum()
            std_pnl   = pnl_series.std(ddof=0)
            print(f" PnL={total_pnl:.2f}, Std={std_pnl:.2f}")

            if total_pnl > best_pnl:
                best_pnl = total_pnl
                best_std = std_pnl
                best_N   = N
                best_W   = W

        print(f" => Best: N={best_N}, W={best_W}, TotalPnL={best_pnl:.2f}, Std={best_std:.2f}")

        results.append({
            "Instrument": inst,
            "Best_N":      best_N,
            "Best_W":      best_W,
            "TotalPnL":    best_pnl,
            "PnL_Std":     best_std
        })

    df = pd.DataFrame(results)
    pd.set_option("display.float_format", "{:.2f}".format)
    print("\n=== Per-Instrument Grid-Search Results ===")
    print(df.to_string(index=False))


In [ ]:
#!/usr/bin/env python3
"""
pip_trader_with_plot_individual_and_metrics.py
─────────────────────────────────────────
Standalone script that:

1. Implements the dynamic PIP trader exactly as before.
2. Loads `prices.txt`, runs the trader on 50 instruments.
3. Plots each instrument in individual figures with:
   • price series (black) + green/red shading for long/short
   • cumulative PnL (below price)
   • confidence level (below PnL)
   • takes a commission of 0.0005 per dollar traded
"""
from pathlib import Path
from typing    import List, Optional, Tuple

import numpy as np
import matplotlib.pyplot as plt


# ──────────────────── Trader scaffolding ─────────────────────
def export(fn):
    """No-op decorator to mark exported methods."""
    fn._is_exported = True
    return fn

class Trader:
    """Minimal base class for our model."""
    def Alg(self, prcSoFar: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
        raise NotImplementedError


# ────────────────────── strategy hyper‐parameters ───────────────────
BEST_N: List[int] = [
    200, 200, 400, 200, 400, 400, 100, 300, 100, 200,
    100, 500, 100, 300, 200, 100, 200, 100, 200, 300,
    100, 200, 500, 100, 400, 100, 100, 400, 100, 100,
    400, 500, 300, 200, 400, 500, 100, 400, 100, 100,
    100, 300, 400, 500, 100, 400, 500, 100, 500, 300,
]
BEST_W: List[int] = [
     10,  10, 100,  10, 100,  10, 100,  25,  10, 100,
    100,  50,  25,  10,  50,  25,  10,  25,  50, 100,
    100,  10,  10,  50,  25,  25,  10,  50,  25, 100,
     25,  10, 100,  50,  25,  10,  50, 100,  50,  25,
     25,  10, 100, 100,  50,  50,  10,  25,  25,  25,
]

POSITION_LIMIT   = 10_000
THR_MIN, THR_MAX, THR_STEP = 0.0, 0.700, 0.0005
THR_GRID         = np.arange(THR_MIN, THR_MAX + THR_STEP, THR_STEP, dtype=float)
COMMISSION_RATE  = 0.0005  # per dollar traded


# ────────────────────── helper functions ─────────────────────────
def causal_update(px: float,
                  last_extreme: float,
                  direction: Optional[int],
                  thr: float) -> Tuple[int, float]:
    """
    One‐step causal PIP update.
    Returns (new_direction, new_last_extreme).
    """
    if direction == 0 or direction is None:
        move = (px - last_extreme) / last_extreme
        if abs(move) >= thr:
            return (1 if move > 0 else -1), px
        return 0, last_extreme
    if direction == 1:  # up‐swing
        if px > last_extreme:
            return 1, px
        if (last_extreme - px) / last_extreme >= thr:
            return -1, px
        return 1, last_extreme
    # direction == -1 (down‐swing)
    if px < last_extreme:
        return -1, px
    if (px - last_extreme) / last_extreme >= thr:
        return 1, px
    return -1, last_extreme


def best_threshold(window: np.ndarray) -> float:
    """
    Overflow‐safe grid‐search of pivot threshold on `window`.
    """
    best_thr, best_pnl = THR_GRID[0], -np.inf
    for thr in THR_GRID:
        dir_vec = np.zeros(len(window), dtype=np.int8)
        le, d = window[0], 0
        for k in range(1, len(window)):
            d, le = causal_update(window[k], le, d, thr)
            dir_vec[k] = d
        pnl = np.sum(dir_vec[:-1].astype(np.int32) *
                     POSITION_LIMIT *
                     np.diff(window))
        if pnl > best_pnl:
            best_pnl, best_thr = pnl, thr
    return best_thr


# ───────────────────────── trader class ───────────────────────────
class CausalPIPTrader(Trader):
    """
    Per‐instrument rolling‐window PIP trader with confidence outputs.
    """

    class _InstrState:
        __slots__ = ("n", "w", "next_calib", "thr",
                     "dir", "last_extreme")

        def __init__(self, n: int, w: int):
            self.n = n
            self.w = w
            self.next_calib = n      # first calibration at bar N
            self.thr = None          # threshold once fit
            self.dir = 0            # current direction
            self.last_extreme = None

    def __init__(self):
        super().__init__()
        if len(BEST_N) != 50 or len(BEST_W) != 50:
            raise ValueError("Need exactly 50 (N, W) pairs.")
        self._states = [self._InstrState(n, w)
                        for n, w in zip(BEST_N, BEST_W)]
        print("CausalPIPTrader initialized.")

    @export
    def Alg(self, prcSoFar: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
        nInst, t_seen = prcSoFar.shape
        if nInst != 50:
            raise ValueError("Expecting 50 instruments.")
        positions = np.zeros(nInst, dtype=np.int32)
        confidences = np.zeros(nInst, dtype=float)

        for i in range(nInst):
            st = self._states[i]
            price = prcSoFar[i, -1]

            # initialize last extreme
            if st.last_extreme is None:
                st.last_extreme = price

            # recalibrate threshold when due
            if t_seen >= st.next_calib:
                window = prcSoFar[i, t_seen - st.n : t_seen]
                st.thr = best_threshold(window)
                st.next_calib += st.w
                print(f"[t={t_seen:4d}] Inst {i:2d} thr={st.thr:.4f}")

            # skip until first calibration
            if st.thr is None:
                continue

            # record prior extreme for confidence calc
            prev_extreme = st.last_extreme

            # one‐step causal update
            new_dir, new_extreme = causal_update(
                price, prev_extreme, st.dir, st.thr
            )
            # compute confidence as relative move / thr
            move_ratio = abs((price - prev_extreme) / prev_extreme) / st.thr if st.thr > 0 else 0.0
            confidences[i] = min(move_ratio, 1.0)

            st.dir, st.last_extreme = new_dir, new_extreme
            positions[i] = new_dir * POSITION_LIMIT

        return positions, confidences


# ──────────────────── run and cache outputs ────────────────────────
prices = np.loadtxt(Path("prices.txt"), delimiter=None)  # shape (T,50)
T, nInst = prices.shape
trader = CausalPIPTrader()
_cached_positions = np.zeros((T, nInst), dtype=int)
_cached_confidences = np.zeros((T, nInst), dtype=float)

for t in range(1, T + 1):
    pos, conf = trader.Alg(prices[:t, :].T)
    _cached_positions[t-1] = pos
    _cached_confidences[t-1] = conf

print("Finished running trader and caching positions + confidences.")

# ─────────────────── calculate PnL and cumulative with commission ──────────────────

pnl = np.zeros((T, nInst), dtype=float)
for t in range(1, T):
    prev_pos    = _cached_positions[t-1]
    curr_pos    = _cached_positions[t]
    price_diff  = prices[t] - prices[t-1]
    trade_amount= np.abs(curr_pos - prev_pos) * prices[t]
    pnl[t]      = prev_pos * price_diff - COMMISSION_RATE * trade_amount

cumulative_pnl = np.cumsum(pnl, axis=0)

# ───────────────────── plotting per instrument ─────────────────────
for i in range(nInst):
    times    = np.arange(T)
    series   = prices[:, i]
    pos      = _cached_positions[:, i]
    cum_pnl  = cumulative_pnl[:, i]
    conf     = _cached_confidences[:, i]

    fig, axs = plt.subplots(
        3, 1,
        figsize=(10, 12),
        sharex=True,
        gridspec_kw={"height_ratios": [1, 1, 1]}
    )
    ax_price, ax_pnl, ax_conf = axs

    # price + shading
    ax_price.plot(times, series, linewidth=1)
    ax_price.fill_between(
        times, series.min(), series,
        where=pos > 0, facecolor="green", alpha=0.3, step="mid"
    )
    ax_price.fill_between(
        times, series.max(), series,
        where=pos < 0, facecolor="red", alpha=0.3, step="mid"
    )
    ax_price.set_ylabel("Price")

    # cumulative PnL
    ax_pnl.plot(times, cum_pnl, linewidth=1)
    ax_pnl.set_ylabel("Cumulative PnL")
    ax_pnl.axhline(0, linewidth=0.5)

    # confidence
    ax_conf.plot(times, conf, linewidth=1)
    ax_conf.set_ylabel("Confidence")
    ax_conf.set_xlabel("Timestep")
    ax_conf.set_ylim(0, 1)

    fig.suptitle(f"Instrument {i:02d}")
    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    plt.show()


In [ ]:
#!/usr/bin/env python3
"""
pip_trader_with_plot_individual_and_metrics.py
─────────────────────────────────────────
Standalone script that:

1. Implements the dynamic PIP trader exactly as before.
2. Loads `prices.txt`, runs the trader on 50 instruments.
3. Plots each instrument in individual figures with:
   • price series (black) + green/red shading for long/short
   • cumulative PnL (below price)
   • confidence level (below PnL)
   • takes a commission of 0.0005 per dollar traded
"""
from pathlib import Path
from typing    import List, Optional, Tuple

import numpy as np
import matplotlib.pyplot as plt


# ──────────────────── Trader scaffolding ─────────────────────
def export(fn):
    """No-op decorator to mark exported methods."""
    fn._is_exported = True
    return fn

class Trader:
    """Minimal base class for our model."""
    def Alg(self, prcSoFar: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
        raise NotImplementedError


# ────────────────────── strategy hyper‐parameters ───────────────────
BEST_N: List[int] = [
    200, 200, 400, 200, 400, 400, 100, 300, 100, 200,
    100, 500, 100, 300, 200, 100, 200, 100, 200, 300,
    100, 200, 500, 100, 400, 100, 100, 400, 100, 100,
    400, 500, 300, 200, 400, 500, 100, 400, 100, 100,
    100, 300, 400, 500, 100, 400, 500, 100, 500, 300,
]
BEST_W: List[int] = [
     10,  10, 100,  10, 100,  10, 100,  25,  10, 100,
    100,  50,  25,  10,  50,  25,  10,  25,  50, 100,
    100,  10,  10,  50,  25,  25,  10,  50,  25, 100,
     25,  10, 100,  50,  25,  10,  50, 100,  50,  25,
     25,  10, 100, 100,  50,  50,  10,  25,  25,  25,
]

POSITION_LIMIT   = 10_000
THR_MIN, THR_MAX, THR_STEP = 0.0, 0.700, 0.0005
THR_GRID         = np.arange(THR_MIN, THR_MAX + THR_STEP, THR_STEP, dtype=float)
COMMISSION_RATE  = 0.000  # per dollar traded


# ────────────────────── helper functions ─────────────────────────
def causal_update(px: float,
                  last_extreme: float,
                  direction: Optional[int],
                  thr: float) -> Tuple[int, float]:
    """
    One‐step causal PIP update.
    Returns (new_direction, new_last_extreme).
    """
    if direction == 0 or direction is None:
        move = (px - last_extreme) / last_extreme
        if abs(move) >= thr:
            return (1 if move > 0 else -1), px
        return 0, last_extreme
    if direction == 1:  # up‐swing
        if px > last_extreme:
            return 1, px
        if (last_extreme - px) / last_extreme >= thr:
            return -1, px
        return 1, last_extreme
    # direction == -1 (down‐swing)
    if px < last_extreme:
        return -1, px
    if (px - last_extreme) / last_extreme >= thr:
        return 1, px
    return -1, last_extreme


def best_threshold(window: np.ndarray) -> float:
    """
    Overflow‐safe grid‐search of pivot threshold on `window`.
    """
    best_thr, best_pnl = THR_GRID[0], -np.inf
    for thr in THR_GRID:
        dir_vec = np.zeros(len(window), dtype=np.int8)
        le, d = window[0], 0
        for k in range(1, len(window)):
            d, le = causal_update(window[k], le, d, thr)
            dir_vec[k] = d
        pnl = np.sum(dir_vec[:-1].astype(np.int32) *
                     POSITION_LIMIT *
                     np.diff(window))
        if pnl > best_pnl:
            best_pnl, best_thr = pnl, thr
    return best_thr


# ───────────────────────── trader class ───────────────────────────
class CausalPIPTrader(Trader):
    """
    Per‐instrument rolling‐window PIP trader with confidence outputs.
    """

    class _InstrState:
        __slots__ = ("n", "w", "next_calib", "thr",
                     "dir", "last_extreme")

        def __init__(self, n: int, w: int):
            self.n = n
            self.w = w
            self.next_calib = n      # first calibration at bar N
            self.thr = None          # threshold once fit
            self.dir = 0            # current direction
            self.last_extreme = None

    def __init__(self):
        super().__init__()
        if len(BEST_N) != 50 or len(BEST_W) != 50:
            raise ValueError("Need exactly 50 (N, W) pairs.")
        self._states = [self._InstrState(n, w)
                        for n, w in zip(BEST_N, BEST_W)]
        print("CausalPIPTrader initialized.")

    @export
    def Alg(self, prcSoFar: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
        nInst, t_seen = prcSoFar.shape
        if nInst != 50:
            raise ValueError("Expecting 50 instruments.")
        positions = np.zeros(nInst, dtype=np.int32)
        confidences = np.zeros(nInst, dtype=float)

        for i in range(nInst):
            st = self._states[i]
            price = prcSoFar[i, -1]

            # initialize last extreme
            if st.last_extreme is None:
                st.last_extreme = price

            # recalibrate threshold when due
            if t_seen >= st.next_calib:
                window = prcSoFar[i, t_seen - st.n : t_seen]
                st.thr = best_threshold(window)
                st.next_calib += st.w
                print(f"[t={t_seen:4d}] Inst {i:2d} thr={st.thr:.4f}")

            # skip until first calibration
            if st.thr is None:
                continue

            # record prior extreme for confidence calc
            prev_extreme = st.last_extreme

            # one‐step causal update
            new_dir, new_extreme = causal_update(
                price, prev_extreme, st.dir, st.thr
            )
            # compute confidence as relative move / thr
            move_ratio = abs((price - prev_extreme) / prev_extreme) / st.thr if st.thr > 0 else 0.0
            confidences[i] = min(move_ratio, 1.0)

            st.dir, st.last_extreme = new_dir, new_extreme
            positions[i] = new_dir * POSITION_LIMIT

        return positions, confidences


# ──────────────────── run and cache outputs ────────────────────────
prices = np.loadtxt(Path("prices.txt"), delimiter=None)  # shape (T,50)
T, nInst = prices.shape
trader = CausalPIPTrader()
_cached_positions = np.zeros((T, nInst), dtype=int)
_cached_confidences = np.zeros((T, nInst), dtype=float)

for t in range(1, T + 1):
    pos, conf = trader.Alg(prices[:t, :].T)
    _cached_positions[t-1] = pos
    _cached_confidences[t-1] = conf

print("Finished running trader and caching positions + confidences.")

# ─────────────────── calculate PnL and cumulative with commission ──────────────────

pnl = np.zeros((T, nInst), dtype=float)
for t in range(1, T):
    prev_pos    = _cached_positions[t-1]
    curr_pos    = _cached_positions[t]
    price_diff  = prices[t] - prices[t-1]
    trade_amount= np.abs(curr_pos - prev_pos) * prices[t]
    pnl[t]      = prev_pos * price_diff - COMMISSION_RATE * trade_amount

cumulative_pnl = np.cumsum(pnl, axis=0)

# ───────────────────── plotting per instrument ─────────────────────
for i in range(nInst):
    times    = np.arange(T)
    series   = prices[:, i]
    pos      = _cached_positions[:, i]
    cum_pnl  = cumulative_pnl[:, i]
    conf     = _cached_confidences[:, i]

    fig, axs = plt.subplots(
        3, 1,
        figsize=(10, 12),
        sharex=True,
        gridspec_kw={"height_ratios": [1, 1, 1]}
    )
    ax_price, ax_pnl, ax_conf = axs

    # price + shading
    ax_price.plot(times, series, linewidth=1)
    ax_price.fill_between(
        times, series.min(), series,
        where=pos > 0, facecolor="green", alpha=0.3, step="mid"
    )
    ax_price.fill_between(
        times, series.max(), series,
        where=pos < 0, facecolor="red", alpha=0.3, step="mid"
    )
    ax_price.set_ylabel("Price")

    # cumulative PnL
    ax_pnl.plot(times, cum_pnl, linewidth=1)
    ax_pnl.set_ylabel("Cumulative PnL")
    ax_pnl.axhline(0, linewidth=0.5)

    # confidence
    ax_conf.plot(times, conf, linewidth=1)
    ax_conf.set_ylabel("Confidence")
    ax_conf.set_xlabel("Timestep")
    ax_conf.set_ylim(0, 1)

    fig.suptitle(f"Instrument {i:02d}")
    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    plt.show()


In [ ]:
#!/usr/bin/env python3
"""
stoploss_gridsearch_pip_trader.py  – FIX v2
──────────────────────────────────
* Fixes **IndexError** in `atr_stop` by ensuring the 14‑period ATR
  vector is exactly length T.
* Uses a fully vectorised ATR calculation with `pandas.rolling`.
* No other logic changed – grid‑search still runs Baseline + five
  stop‑loss overlays instrument‑by‑instrument.
"""
import numpy as np
import pandas as pd
from pathlib import Path
from typing import Dict, List
from collections import deque
np.set_printoptions(suppress=True, linewidth=140)

# ══════════════════════ DATA & CONSTANTS ═══════════════════════════
PX         = np.loadtxt(Path("prices.txt"))          # shape (T, 50)
T, N_INST  = PX.shape
COMMISSION = 0.0005
CAPITAL    = 10_000

# ══════════════════════ UTILS ══════════════════════════════════════

def pnl_from_positions(pos: np.ndarray, price: np.ndarray) -> float:
    trade = (pos[:-1] * np.diff(price)).sum()
    turns = np.abs(np.diff(pos))
    comm  = (COMMISSION * turns * price[1:]).sum()
    return trade - comm

# ══════════════════════ RAW BASE MODEL (vectorised) ═══════════════=
class IncWMA:
    def __init__(self,n:int):
        self.n=n; self.buf=deque(maxlen=n); self.w=np.arange(1,n+1); self.S=self.w.sum()
    def update(self,x):
        self.buf.append(x)
        if len(self.buf)<self.n: return x
        arr=np.fromiter(self.buf,float,len(self.buf))
        return float((self.w*arr).sum()/self.S)
class IncHMA:
    def __init__(self,n:int):
        self.f=IncWMA(n); self.h=IncWMA(max(1,n//2)); self.fin=IncWMA(max(1,int(np.sqrt(n))))
    def update(self,p):
        return self.fin.update(2*self.h.update(p)-self.f.update(p))
class IncKalman:
    def __init__(self,R=0.075,Ql=4e-3,Qt=1e-5):
        self.F=np.array([[1,1],[0,1]]); self.H=np.array([[1,0]]); self.Q=np.diag([Ql,Qt]); self.R=R
        self.s=None; self.P=np.eye(2)
    def update(self,x):
        if self.s is None:
            self.s=np.array([x,0.]); return x
        self.s=self.F@self.s; self.P=self.F@self.P@self.F.T+self.Q
        y=x-(self.H@self.s)[0]; S=(self.H@self.P@self.H.T)[0,0]+self.R; K=(self.P@self.H.T)/S
        self.s+=K.flatten()*y; self.P=(np.eye(2)-K@self.H)@self.P; return self.s[0]
class BaseModel:
    def __init__(self,hma_period=100):
        self.h=[IncHMA(hma_period) for _ in range(N_INST)]
        self.k=[IncKalman()       for _ in range(N_INST)]
        self.sig=np.zeros(N_INST,int); self.pos=np.zeros(N_INST,int)
    def step(self,row):
        out=self.pos.copy()
        for i,p in enumerate(row):
            h=self.h[i].update(p); k=self.k[i].update(p); sig=1 if h>k else -1
            if sig!=self.sig[i]:
                out[i]=-sig*int(CAPITAL//p); self.sig[i]=sig; self.pos[i]=out[i]
        return out

def compute_raw_positions():
    bm=BaseModel(); out=np.zeros((T,N_INST),int)
    for t in range(T): out[t]=bm.step(PX[t])
    return out
RAW_POS=compute_raw_positions()
BASELINE=np.array([pnl_from_positions(RAW_POS[:,i],PX[:,i]) for i in range(N_INST)])

# ══════════════════════ STOP‑LOSS OVERLAYS ═════════════════════════

def pct_stop(px,pos,pct):
    out=pos.copy(); entry=0.; dir=0
    for t in range(T):
        if pos[t]==0: dir=0
        elif dir==0:  dir=np.sign(pos[t]); entry=px[t]
        elif (dir>0 and px[t]<=entry*(1-pct)) or (dir<0 and px[t]>=entry*(1+pct)):
            out[t:]=0; break
    return out

# --- FIXED ATR (length T) ---

def atr14(px):
    tr = np.abs(np.diff(px, prepend=px[0]))
    return pd.Series(tr).rolling(14, min_periods=1).mean().values  # length T

def atr_stop(px,pos,k):
    atr=atr14(px); out=pos.copy(); entry=0.; dir=0
    for t in range(T):
        if pos[t]==0: dir=0
        elif dir==0:  dir=np.sign(pos[t]); entry=px[t]
        else:
            thr=k*atr[t]
            if dir>0 and px[t]<=entry-thr: out[t:]=0; break
            if dir<0 and px[t]>=entry+thr: out[t:]=0; break
    return out

# Trailing‑% stop

def trail_stop(px,pos,pct):
    out=pos.copy(); dir=0; peak=trough=0.
    for t in range(T):
        if pos[t]==0: dir=0
        elif dir==0:
            dir=np.sign(pos[t]); peak=trough=px[t]
        else:
            if dir>0:
                peak=max(peak,px[t])
                if px[t]<=peak*(1-pct): out[t:]=0; break
            else:
                trough=min(trough,px[t])
                if px[t]>=trough*(1+pct): out[t:]=0; break
    return out

# Time‑based stop

def time_stop(px,pos,max_bars):
    out=pos.copy(); dur=0
    for t in range(T):
        if pos[t]==0: dur=0
        else:
            dur+=1
            if dur>=max_bars: out[t:]=0; break
    return out

# Volatility‑scaled stop

def vol_stop(px,pos,k):
    sigma=pd.Series(np.diff(px, prepend=px[0])).rolling(20, min_periods=1).std().values
    out=pos.copy(); dir=0; entry=0.
    for t in range(T):
        if pos[t]==0: dir=0
        elif dir==0:  dir=np.sign(pos[t]); entry=px[t]
        else:
            thr=k*sigma[t]
            if dir>0 and px[t]<=entry-thr: out[t:]=0; break
            if dir<0 and px[t]>=entry+thr: out[t:]=0; break
    return out

STOP_FUNCS={
    'Baseline': lambda px,pos,p: pos,
    'PctSL':    pct_stop,
    'ATRSL':    atr_stop,
    'TrailPct': trail_stop,
    'TimeExit': time_stop,
    'VolScaled':vol_stop,
}
GRIDS={
    'Baseline': [0],
    'PctSL':    np.linspace(0.01,0.05,5),
    'ATRSL':    np.arange(1.0,3.1,0.5),
    'TrailPct': np.linspace(0.02,0.08,4),
    'TimeExit': np.arange(50,251,50),
    'VolScaled':np.arange(1.0,3.1,0.5),
}

best_param={k:[None]*N_INST for k in GRIDS}
best_pnl  ={k:[-1e30]*N_INST for k in GRIDS}

print("=== Grid search start ===")
for name, grid in GRIDS.items():
    f=STOP_FUNCS[name]
    for inst in range(N_INST):
        px=PX[:,inst]; raw=RAW_POS[:,inst]
        for g in grid:
            pnl=pnl_from_positions(f(px,raw,g),px)
            if pnl>best_pnl[name][inst]:
                best_pnl[name][inst]=pnl; best_param[name][inst]=g
        print(f"{name:8s} inst {inst:02d} bestPnL={best_pnl[name][inst]:.0f} param={best_param[name][inst]}")
print("=== Grid search complete ===\n")

# ══════════════════════ SUMMARY ═══════════════════════════════════
print("Total PnL per overlay (sum over 50 instruments):")
for name in GRIDS:
    total=sum(best_pnl[name]) if name!='Baseline' else BASELINE.sum()
    print(f"{name:8s}  {total:,.0f}")


In [ ]:
#!/usr/bin/env python3
"""
plot_timeexit_vs_baseline.py
────────────────────────────
Plots Baseline vs. Time‑Exit cumulative PnL for each of the 50 instruments.

•  Re‑implements the Baseline model exactly as in
   stoploss_gridsearch_pip_trader.py (HMA‑Kalman flip model).
•  Uses your instrument‑specific TimeExit parameters.
•  Saves/Shows one figure per instrument with two curves:
   – Baseline cumulative PnL
   – TimeExit cumulative PnL
"""

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from collections import deque

# ────────────────── CONFIG ────────────────────────────────────────────
PRICE_FILE  = Path("prices.txt")       # shape (T, 50)  already transposed
COMMISSION  = 0.0005
CAPITAL     = 10_000
SAVE_PLOTS  = False                    # True → PNGs in ./plots/

# Instrument‑specific TimeExit (max_bars) values (index = instrument id)
TIMEEXIT_PARAMS = [
    200,100, 50, 50, 50, 50,100, 50,250,250,
    200,250,250,250,100, 50, 50,250, 50,100,
    250, 50, 50,150,150,150,250, 50,250, 50,
    250,200,250,150, 50, 50,200, 50, 50,200,
    100,250,150,150,250,100,150, 50,100,200
]

# ────────────────── LOAD DATA ──────────────────────────────────────────
PX = np.loadtxt(PRICE_FILE)            # (T, 50)
T, N_INST = PX.shape
assert len(TIMEEXIT_PARAMS) == N_INST, "Need 50 TimeExit parameters!"

# ────────────────── HELPERS ────────────────────────────────────────────
def cumulative_pnl(pos: np.ndarray, price: np.ndarray) -> np.ndarray:
    """
    Vectorised cumulative PnL series (same commission model as before).
    Returns array of length T (cumPnL[0] = 0).
    """
    trade = pos[:-1] * np.diff(price)                       # shape (T-1,)
    turns = np.abs(np.diff(pos)) * price[1:] * COMMISSION   # commission $
    pnl_step = trade - turns
    return np.concatenate(([0.0], np.cumsum(pnl_step)))

# ────────────────── BASE MODEL (unchanged) ────────────────────────────
class IncWMA:
    def __init__(self, n: int):
        self.n = n
        self.buf = deque(maxlen=n)
        self.w = np.arange(1, n + 1, dtype=float)
        self.S = self.w.sum()
    def update(self, x: float) -> float:
        self.buf.append(x)
        if len(self.buf) < self.n:
            return x
        arr = np.fromiter(self.buf, float, len(self.buf))
        return float((self.w * arr).sum() / self.S)

class IncHMA:
    def __init__(self, n: int):
        self.full = IncWMA(n)
        self.half = IncWMA(max(1, n // 2))
        self.fin  = IncWMA(max(1, int(np.sqrt(n))))
    def update(self, p: float) -> float:
        return self.fin.update(2 * self.half.update(p) - self.full.update(p))

class IncKalman:
    def __init__(self, R=0.075, Ql=4e-3, Qt=1e-5):
        self.F = np.array([[1, 1], [0, 1]])
        self.H = np.array([[1, 0]])
        self.Q = np.diag([Ql, Qt])
        self.R = R
        self.s = None
        self.P = np.eye(2)
    def update(self, x: float) -> float:
        if self.s is None:
            self.s = np.array([x, 0.0])
            return x
        self.s = self.F @ self.s
        self.P = self.F @ self.P @ self.F.T + self.Q
        y = x - (self.H @ self.s)[0]
        S = (self.H @ self.P @ self.H.T)[0, 0] + self.R
        K = (self.P @ self.H.T) / S
        self.s += (K.flatten() * y)
        self.P = (np.eye(2) - K @ self.H) @ self.P
        return self.s[0]

class BaseModel:
    def __init__(self, hma_period=100):
        self.h = [IncHMA(hma_period) for _ in range(N_INST)]
        self.k = [IncKalman()        for _ in range(N_INST)]
        self.sig = np.zeros(N_INST, int)
        self.pos = np.zeros(N_INST, int)
    def step(self, prices_row: np.ndarray) -> np.ndarray:
        out = self.pos.copy()
        for i, px in enumerate(prices_row):
            h = self.h[i].update(px)
            k = self.k[i].update(px)
            sig = 1 if h > k else -1
            if sig != self.sig[i]:
                out[i] = -sig * int(CAPITAL // px)
                self.sig[i] = sig
                self.pos[i] = out[i]
        return out

def compute_baseline_positions() -> np.ndarray:
    bm = BaseModel()
    out = np.zeros((T, N_INST), dtype=int)
    for t in range(T):
        out[t] = bm.step(PX[t])
    return out

# ────────────────── TimeExit overlay ───────────────────────────────────
def time_exit_overlay(price: np.ndarray,
                      raw_pos: np.ndarray,
                      max_bars: int) -> np.ndarray:
    out = raw_pos.copy()
    duration = 0
    for t in range(T):
        if raw_pos[t] == 0:
            duration = 0
        else:
            duration += 1
            if duration >= max_bars:
                out[t:] = 0
                break
    return out

# ────────────────── MAIN COMPUTATION ───────────────────────────────────
print("Computing Baseline positions …")
RAW_POS = compute_baseline_positions()

print("Applying TimeExit overlays …")
TIMEEXIT_POS = np.zeros_like(RAW_POS, dtype=int)
for i in range(N_INST):
    TIMEEXIT_POS[:, i] = time_exit_overlay(PX[:, i],
                                           RAW_POS[:, i],
                                           TIMEEXIT_PARAMS[i])

# ────────────────── PnL SERIES ─────────────────────────────────────────
print("Computing cumulative PnL series …")
cum_baseline = np.zeros_like(RAW_POS, dtype=float)
cum_timeexit = np.zeros_like(RAW_POS, dtype=float)

for i in range(N_INST):
    cum_baseline[:, i] = cumulative_pnl(RAW_POS[:, i], PX[:, i])
    cum_timeexit[:, i] = cumulative_pnl(TIMEEXIT_POS[:, i], PX[:, i])

# ────────────────── PLOTTING ───────────────────────────────────────────
print("Plotting …")
if SAVE_PLOTS:
    Path("plots").mkdir(exist_ok=True)

for i in range(N_INST):
    plt.figure(figsize=(10, 5))
    plt.plot(cum_baseline[:, i],
             label="Baseline",
             linewidth=1.4)
    plt.plot(cum_timeexit[:, i],
             label=f"TimeExit (max={TIMEEXIT_PARAMS[i]})",
             linewidth=1.4)
    plt.title(f"Instrument {i:02d} – Cumulative PnL")
    plt.xlabel("Time step")
    plt.ylabel("PnL ($)")
    plt.legend()
    plt.grid(True, linestyle="--", alpha=0.4)
    if SAVE_PLOTS:
        plt.savefig(Path("plots") / f"inst_{i:02d}_pnl.png", dpi=150)
        plt.close()
    else:
        plt.show(block=False)   # comment out if you prefer blocking
        plt.pause(0.05)

print("Done.")


In [ ]:
#!/usr/bin/env python3
"""
plot_pctsl_vs_baseline.py
─────────────────────────
Plots Baseline vs. PctSL cumulative PnL for each of the 50 instruments.

•  Re‑implements the Baseline model exactly as in
   stoploss_gridsearch_pip_trader.py (HMA‑Kalman flip model).
•  Applies the Pct‑stop overlay with your instrument‑specific
   percentages (see PCTSL_PARAMS).
•  Shows or saves one figure per instrument:
      – Baseline cumulative PnL
      – PctSL cumulative PnL
"""

from pathlib import Path
from collections import deque
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# ───────────────── CONFIG ──────────────────────────────────────────────
PRICE_FILE = Path("prices.txt")     # shape (T, 50)  already transposed
COMMISSION = 0.0005
CAPITAL    = 10_000
SAVE_PLOTS = False                  # True → PNGs in ./plots/

# Per‑instrument best PctSL (param = stop %)
PCTSL_PARAMS = [
    0.05,0.02,0.03,0.05,0.05,0.05,0.05,0.03,0.01,0.05,
    0.01,0.01,0.05,0.05,0.04,0.05,0.05,0.03,0.05,0.01,
    0.01,0.05,0.05,0.01,0.01,0.05,0.01,0.05,0.03,0.01,
    0.02,0.02,0.04,0.02,0.03,0.01,0.05,0.05,0.05,0.05,
    0.03,0.04,0.05,0.01,0.03,0.04,0.05,0.05,0.04,0.03
]

# ───────────────── LOAD DATA ───────────────────────────────────────────
PX = np.loadtxt(PRICE_FILE)         # (T, 50)
T, N_INST = PX.shape
assert len(PCTSL_PARAMS) == N_INST, "Need 50 PctSL parameters!"

# ───────────────── HELPERS ─────────────────────────────────────────────
def cumulative_pnl(pos: np.ndarray, price: np.ndarray) -> np.ndarray:
    """Vectorised cumulative PnL (Baseline commission model)."""
    trade = pos[:-1] * np.diff(price)
    turns = np.abs(np.diff(pos)) * price[1:] * COMMISSION
    pnl_step = trade - turns
    return np.concatenate(([0.0], np.cumsum(pnl_step)))

# ─────────────── Baseline model (identical to original) ───────────────
class IncWMA:
    def __init__(self, n: int):
        self.n = n
        self.buf = deque(maxlen=n)
        self.w = np.arange(1, n + 1, dtype=float)
        self.S = self.w.sum()
    def update(self, x: float) -> float:
        self.buf.append(x)
        if len(self.buf) < self.n:
            return x
        arr = np.fromiter(self.buf, float, len(self.buf))
        return float((self.w * arr).sum() / self.S)

class IncHMA:
    def __init__(self, n: int):
        self.full = IncWMA(n)
        self.half = IncWMA(max(1, n // 2))
        self.fin  = IncWMA(max(1, int(np.sqrt(n))))
    def update(self, p: float) -> float:
        return self.fin.update(2 * self.half.update(p) - self.full.update(p))

class IncKalman:
    def __init__(self, R=0.075, Ql=4e-3, Qt=1e-5):
        self.F = np.array([[1,1],[0,1]])
        self.H = np.array([[1,0]])
        self.Q = np.diag([Ql, Qt])
        self.R = R
        self.s = None
        self.P = np.eye(2)
    def update(self, x: float) -> float:
        if self.s is None:
            self.s = np.array([x,0.])
            return x
        self.s = self.F @ self.s
        self.P = self.F @ self.P @ self.F.T + self.Q
        y = x - (self.H @ self.s)[0]
        S = (self.H @ self.P @ self.H.T)[0,0] + self.R
        K = (self.P @ self.H.T) / S
        self.s += (K.flatten() * y)
        self.P = (np.eye(2) - K @ self.H) @ self.P
        return self.s[0]

class BaseModel:
    def __init__(self, hma_period=100):
        self.h = [IncHMA(hma_period) for _ in range(N_INST)]
        self.k = [IncKalman()        for _ in range(N_INST)]
        self.sig = np.zeros(N_INST, int)
        self.pos = np.zeros(N_INST, int)
    def step(self, prices_row: np.ndarray) -> np.ndarray:
        out = self.pos.copy()
        for i, px in enumerate(prices_row):
            h = self.h[i].update(px)
            k = self.k[i].update(px)
            sig = 1 if h > k else -1
            if sig != self.sig[i]:
                out[i] = -sig * int(CAPITAL // px)
                self.sig[i] = sig
                self.pos[i] = out[i]
        return out

def compute_baseline_positions() -> np.ndarray:
    bm = BaseModel()
    out = np.zeros((T, N_INST), dtype=int)
    for t in range(T):
        out[t] = bm.step(PX[t])
    return out

# ──────────────── PctSL overlay ───────────────────────────────────────
def pct_stop_overlay(price: np.ndarray,
                     raw_pos: np.ndarray,
                     pct: float) -> np.ndarray:
    out = raw_pos.copy()
    dirn = 0
    entry = 0.0
    for t in range(T):
        if raw_pos[t] == 0:
            dirn = 0
        elif dirn == 0:
            dirn = np.sign(raw_pos[t])
            entry = price[t]
        else:
            if (dirn > 0 and price[t] <= entry * (1 - pct)) or \
               (dirn < 0 and price[t] >= entry * (1 + pct)):
                out[t:] = 0
                break
    return out

# ───────────────── MAIN COMPUTATION ───────────────────────────────────
print("Computing Baseline positions …")
RAW_POS = compute_baseline_positions()

print("Applying PctSL overlays …")
PCTSL_POS = np.zeros_like(RAW_POS, dtype=int)
for i in range(N_INST):
    PCTSL_POS[:, i] = pct_stop_overlay(PX[:, i],
                                       RAW_POS[:, i],
                                       PCTSL_PARAMS[i])

# ─────────────── PnL SERIES ───────────────────────────────────────────
print("Computing cumulative PnL series …")
cum_baseline = np.zeros_like(RAW_POS, dtype=float)
cum_pctsl    = np.zeros_like(RAW_POS, dtype=float)

for i in range(N_INST):
    cum_baseline[:, i] = cumulative_pnl(RAW_POS[:, i], PX[:, i])
    cum_pctsl[:, i]    = cumulative_pnl(PCTSL_POS[:, i], PX[:, i])

# ───────────────── PLOTTING ───────────────────────────────────────────
print("Plotting …")
if SAVE_PLOTS:
    Path("plots").mkdir(exist_ok=True)

for i in range(N_INST):
    plt.figure(figsize=(10, 5))
    plt.plot(cum_baseline[:, i], label="Baseline", linewidth=1.4)
    plt.plot(cum_pctsl[:, i],
             label=f"PctSL (pct={PCTSL_PARAMS[i]:.2%})",
             linewidth=1.4)
    plt.title(f"Instrument {i:02d} – Cumulative PnL")
    plt.xlabel("Time step")
    plt.ylabel("PnL ($)")
    plt.legend()
    plt.grid(True, linestyle="--", alpha=0.4)
    if SAVE_PLOTS:
        plt.savefig(Path("plots") / f"inst_{i:02d}_pctsl.png", dpi=150)
        plt.close()
    else:
        plt.show(block=False)      # non‑blocking windows
        plt.pause(0.05)

print("Done.")


In [ ]:
#!/usr/bin/env python3
"""
causal_pip_pnl_plot.py
──────────────────────
• Causal PIP trader (same logic as before).
• Detects PIP turning-points on cumulative PnL using a *global* threshold
  and overlays them on the PnL plot.
• Outputs, per instrument, a 2-panel figure:
      (1) price with long/short shading
      (2) cumulative PnL with detected PIPs highlighted
"""

from pathlib import Path
from typing  import List, Optional, Tuple

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


# ═══════════════ strategy constants ════════════════════════════════════ #
BEST_N: List[int] = [
    200, 200, 400, 200, 400, 400, 100, 300, 100, 200,
    100, 500, 100, 300, 200, 100, 200, 100, 200, 300,
    100, 200, 500, 100, 400, 100, 100, 400, 100, 100,
    400, 500, 300, 200, 400, 500, 100, 400, 100, 100,
    100, 300, 400, 500, 100, 400, 500, 100, 500, 300,
]
BEST_W: List[int] = [
     10,  10, 100,  10, 100,  10, 100,  25,  10, 100,
    100,  50,  25,  10,  50,  25,  10,  25,  50, 100,
    100,  10,  10,  50,  25,  25,  10,  50,  25, 100,
     25,  10, 100,  50,  25,  10,  50, 100,  50,  25,
     25,  10, 100, 100,  50,  50,  10,  25,  25,  25,
]

POSITION_LIMIT    = 10_000
COMMISSION_RATE   = 0.0005           # per-dollar commission
THR_MIN, THR_MAX, THR_STEP = 0.00, 0.700, 0.0005
THR_GRID          = np.arange(THR_MIN, THR_MAX + THR_STEP, THR_STEP)

PNL_THR           = 0.10             # PIP threshold on cumulative PnL (10 %)

# ═══════════════ causal-PIP helpers ════════════════════════════════════ #
def causal_update(px: float,
                  last_extreme: float,
                  direction: Optional[int],
                  thr: float) -> Tuple[int, float]:
    """
    One-step causal PIP update.
    Returns (new_direction, new_last_extreme).
    """
    if direction == 0 or direction is None:
        mv = (px - last_extreme) / last_extreme
        if abs(mv) >= thr:
            return (1 if mv > 0 else -1), px
        return 0, last_extreme

    if direction == 1:               # trending up
        if px > last_extreme:
            return 1, px
        if (last_extreme - px) / last_extreme >= thr:
            return -1, px
        return 1, last_extreme

    # direction == -1 (trending down)
    if px < last_extreme:
        return -1, px
    if (px - last_extreme) / last_extreme >= thr:
        return 1, px
    return -1, last_extreme


def best_threshold(window: np.ndarray) -> float:
    """
    Grid-search optimal threshold on `window` for max PnL.

    ✱ Fix: cast `dir_vec` to int32 before multiplying by POSITION_LIMIT
           (avoids int8 overflow).
    """
    best_thr, best_pnl = THR_GRID[0], -np.inf
    for thr in THR_GRID:
        dir_vec = np.zeros(len(window), dtype=np.int8)
        le, d = window[0], 0
        for k in range(1, len(window)):
            d, le = causal_update(window[k], le, d, thr)
            dir_vec[k] = d
        pnl = np.sum(dir_vec[:-1].astype(np.int32) * POSITION_LIMIT * np.diff(window))
        if pnl > best_pnl:
            best_thr, best_pnl = thr, pnl
    return best_thr


# ═══════════════ trader implementation ════════════════════════════════ #
def export(fn):
    """No-op decorator for the competition harness."""
    fn._is_exported = True
    return fn


class Trader:
    """Abstract base."""
    def Alg(self, prcSoFar: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
        raise NotImplementedError


class CausalPIPTrader(Trader):
    """
    Per-instrument PIP trader producing (positions, confidences).
    Threshold grid-search is re-run every W bars.
    """
    class _InstrState:
        __slots__ = ("n", "w", "next_calib", "thr", "dir", "last_extreme")
        def __init__(self, n: int, w: int):
            self.n = n
            self.w = w
            self.next_calib = n
            self.thr: Optional[float] = None
            self.dir = 0
            self.last_extreme: Optional[float] = None

    def __init__(self):
        self._states = [self._InstrState(n, w) for n, w in zip(BEST_N, BEST_W)]
        print("CausalPIPTrader initialised.")

    @export
    def Alg(self, prcSoFar: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
        """
        Parameters
        ----------
        prcSoFar : (nInst, t) ndarray – price history up to *inclusive* t-1.
        """
        nInst, t_seen = prcSoFar.shape
        positions   = np.zeros(nInst, dtype=np.int32)
        confidences = np.zeros(nInst, dtype=float)

        for i in range(nInst):
            st   = self._states[i]
            px   = prcSoFar[i, -1]

            if st.last_extreme is None:
                st.last_extreme = px                     # initialise

            # ── periodic threshold calibration ──────────────────────────
            if t_seen >= st.next_calib:
                window       = prcSoFar[i, t_seen - st.n : t_seen]
                st.thr       = best_threshold(window)
                st.next_calib += st.w
                print(f"[t={t_seen:>4}] Inst {i:02d}  new thr = {st.thr:.4f}")

            if st.thr is None:                           # waiting for first calib
                continue

            prev_ext                = st.last_extreme
            new_dir, new_ext        = causal_update(px, prev_ext, st.dir, st.thr)
            move_ratio              = (abs(px - prev_ext) / prev_ext) / st.thr
            confidences[i]          = min(move_ratio, 1.0)

            st.dir, st.last_extreme = new_dir, new_ext
            positions[i]            = new_dir * POSITION_LIMIT

        return positions, confidences


# ═══════════ utilities: PIP detection on cumulative PnL ════════════════ #
def detect_pips(series: np.ndarray, thr: float) -> Tuple[np.ndarray, np.ndarray]:
    """
    Causal PIP detection on `series` (1-D).
    Returns (idx_array, value_array) marking turning points with `thr`.
    """
    if series.size == 0:
        return np.empty(0, int), np.empty(0, float)

    # start at first non-zero to avoid div/0
    start_idx = next((k for k, v in enumerate(series) if v != 0.0), 0)
    last_ext  = series[start_idx]
    last_idx  = start_idx
    direction = 0

    p_idx: List[int]   = [start_idx]
    p_val: List[float] = [series[start_idx]]

    for k in range(start_idx + 1, len(series)):
        px               = series[k]
        new_dir, new_ext = causal_update(px, last_ext, direction, thr)

        if new_dir != direction:         # turning-point detected
            p_idx.append(last_idx)
            p_val.append(last_ext)

        if new_ext != last_ext:
            last_ext, last_idx = new_ext, k
        direction = new_dir

    # record final extreme
    if last_idx not in p_idx:
        p_idx.append(last_idx)
        p_val.append(last_ext)

    return np.asarray(p_idx, int), np.asarray(p_val, float)


# ═══════════════ main script ═══════════════════════════════════════════ #
def main():
    # ── load price matrix (T × nInst) ──────────────────────────────────
    prices = np.loadtxt(Path("prices.txt"))      # rows = time, cols = instruments
    T, nInst = prices.shape

    # ── run trader ─────────────────────────────────────────────────────
    trader = CausalPIPTrader()
    pos_arr  = np.zeros((T, nInst), dtype=int)
    conf_arr = np.zeros((T, nInst), dtype=float)

    for t in range(1, T + 1):
        pos_arr[t - 1], conf_arr[t - 1] = trader.Alg(prices[:t, :].T)

    # ── compute per-step PnL ──────────────────────────────────────────
    trade_pnl = np.zeros((T, nInst), float)
    comm_pnl  = np.zeros((T, nInst), float)

    for t in range(1, T):
        prev_pos, curr_pos = pos_arr[t - 1], pos_arr[t]
        diff               = prices[t] - prices[t - 1]

        trade_pnl[t] = prev_pos * diff
        comm_pnl[t]  = -COMMISSION_RATE * np.abs(curr_pos - prev_pos) * prices[t]

    pnl_total = trade_pnl + comm_pnl
    cum_pnl   = np.cumsum(pnl_total, axis=0)

    # ── plotting per instrument ───────────────────────────────────────
    times = np.arange(T)

    for inst in range(nInst):
        pip_idx, pip_val = detect_pips(cum_pnl[:, inst], PNL_THR)

        fig, (ax_price, ax_pnl) = plt.subplots(
            2, 1, figsize=(10, 8), sharex=True,
            gridspec_kw={"height_ratios": [2, 1]},
        )

        # 1) Price with position shading
        ax_price.plot(times, prices[:, inst], linewidth=1, color="black")
        ax_price.fill_between(
            times, prices[:, inst].min(), prices[:, inst],
            where=pos_arr[:, inst] > 0, facecolor="green", alpha=0.3, step="mid",
        )
        ax_price.fill_between(
            times, prices[:, inst].max(), prices[:, inst],
            where=pos_arr[:, inst] < 0, facecolor="red",   alpha=0.3, step="mid",
        )
        ax_price.set_ylabel("Price")
        ax_price.set_title(f"Instrument {inst:02d}")

        # 2) Cumulative PnL with PIPs
        ax_pnl.plot(times, cum_pnl[:, inst], linewidth=1)
        ax_pnl.axhline(0, color="black", linewidth=0.5)
        ax_pnl.scatter(
            pip_idx, pip_val,
            s=28, marker="o", facecolors="none", edgecolors="blue", linewidths=1.2,
            label=f"PIPs (thr={PNL_THR:.0%})",
        )
        ax_pnl.set_ylabel("Cumulative PnL")
        ax_pnl.set_xlabel("Time")
        ax_pnl.legend(frameon=False)

        plt.tight_layout()
        plt.show()


# ═══════════════════════════════════════════════════════════════════════ #
if __name__ == "__main__":
    main()


In [ ]:
#!/usr/bin/env python3
"""
hybrid_pip_trader_gate_plot.py  (v3)
────────────────────────────────────
• Causal PIP trader  →  suggests positions every bar.
• Causal PnL gate    →  decides (with a 1-bar delay) whether trading is
  enabled (“GREEN”) or disabled (“RED”) based on realised cumulative PnL.
• PIP realised cumulative PnL panel added.

Trading rule
------------
real_pos[t] = suggested_pos[t]   if  gate says GREEN at bar t
            = 0                 otherwise

Key fixes vs previous version
-----------------------------
1. Gate is updated with `cum_real[t-1]` **before** it decides for bar *t*  
   → only a **one-bar** lag, not two.
2. Confidence panel removed.
3. Price shading uses the gate flag, not the suggested position.
4. **PIP realised cumulative PnL** is now plotted in its own panel.
"""

from pathlib import Path
from typing  import List, Optional, Tuple
import numpy as np
import matplotlib.pyplot as plt


# ═════════════ STRATEGY PARAMETERS ════════════════════════════════════ #
BEST_N = [
    200, 200, 400, 200, 400, 400, 100, 300, 100, 200,
    100, 500, 100, 300, 200, 100, 200, 100, 200, 300,
    100, 200, 500, 100, 400, 100, 100, 400, 100, 100,
    400, 500, 300, 200, 400, 500, 100, 400, 100, 100,
    100, 300, 400, 500, 100, 400, 500, 100, 500, 300,
]
BEST_W = [
     10,  10, 100,  10, 100,  10, 100,  25,  10, 100,
    100,  50,  25,  10,  50,  25,  10,  25,  50, 100,
    100,  10,  10,  50,  25,  25,  10,  50,  25, 100,
     25,  10, 100,  50,  25,  10,  50, 100,  50,  25,
     25,  10, 100, 100,  50,  50,  10,  25,  25,  25,
]

POSITION_LIMIT   = 10_000
COMMISSION_RATE  = 0.0005
THR_MIN, THR_MAX, THR_STEP = 0.0, 0.700, 0.0005
THR_GRID = np.arange(THR_MIN, THR_MAX + THR_STEP, THR_STEP)

GATE_THR = 0.10       # 10% swing on cum-PnL flips gate
EPS      = 1e-8


# ═════════════ LOW-LEVEL HELPERS ══════════════════════════════════════ #
def safe_rel_move(a: float, b: float, eps: float = EPS) -> float:
    return (a - b) / max(abs(b), eps)


def pip_update(px: float, last_ext: float,
               direction: Optional[int], thr: float) -> Tuple[int, float]:
    if direction == 0 or direction is None:
        mv = (px - last_ext) / last_ext
        if abs(mv) >= thr:
            return (1 if mv > 0 else -1), px
        return 0, last_ext
    if direction == 1:                      # up-swing
        if px > last_ext:
            return 1, px
        if (last_ext - px) / last_ext >= thr:
            return -1, px
        return 1, last_ext
    # direction == -1
    if px < last_ext:
        return -1, px
    if (px - last_ext) / last_ext >= thr:
        return 1, px
    return -1, last_ext


# ═════════════ TRADER (unchanged logic, confidence ignored) ═══════════ #
class CausalPIPTrader:
    class _S:
        __slots__ = ("n", "w", "next_calib", "thr", "dir", "last_ext")
        def __init__(self, n: int, w: int):
            self.n, self.w = n, w
            self.next_calib = n
            self.thr = None
            self.dir = 0
            self.last_ext = None

    def __init__(self):
        self._st = [self._S(n, w) for n, w in zip(BEST_N, BEST_W)]

    def _best_thr(self, window: np.ndarray) -> float:
        best_thr, best_pnl = THR_GRID[0], -np.inf
        for thr in THR_GRID:
            dvec = np.zeros(len(window), np.int8)
            le, d = window[0], 0
            for k in range(1, len(window)):
                d, le = pip_update(window[k], le, d, thr)
                dvec[k] = d
            pnl = np.sum(dvec[:-1].astype(np.int32) *
                         POSITION_LIMIT *
                         np.diff(window))
            if pnl > best_pnl:
                best_pnl, best_thr = pnl, thr
        return best_thr

    def Alg(self, prc_so_far: np.ndarray) -> np.ndarray:
        nInst, t_seen = prc_so_far.shape
        pos = np.zeros(nInst, np.int32)

        for i in range(nInst):
            st, px = self._st[i], prc_so_far[i, -1]

            if st.last_ext is None:
                st.last_ext = px

            if t_seen >= st.next_calib:
                st.thr = self._best_thr(prc_so_far[i, t_seen - st.n:t_seen])
                st.next_calib += st.w

            if st.thr is None:
                continue

            new_dir, new_ext = pip_update(px, st.last_ext, st.dir, st.thr)
            st.dir, st.last_ext = new_dir, new_ext
            pos[i] = new_dir * POSITION_LIMIT

        return pos


# ═════════════ GATE (causal PnL swing detector) ═══════════════════════ #
class GateState:
    __slots__ = ("direction", "last_ext")
    def __init__(self):
        self.direction: Optional[str] = None
        self.last_ext = 0.0

    def update(self, pnl_now: float) -> None:
        move = safe_rel_move(pnl_now, self.last_ext)
        if self.direction is None:
            if abs(move) >= GATE_THR:
                self.direction = 'up' if move > 0 else 'down'
                self.last_ext  = pnl_now
            return

        if self.direction == 'up':
            if pnl_now > self.last_ext:
                self.last_ext = pnl_now
            elif safe_rel_move(pnl_now, self.last_ext) <= -GATE_THR:
                self.direction = 'down'
                self.last_ext = pnl_now
        else:  # down
            if pnl_now < self.last_ext:
                self.last_ext = pnl_now
            elif safe_rel_move(pnl_now, self.last_ext) >= GATE_THR:
                self.direction = 'up'
                self.last_ext = pnl_now

    @property
    def allow(self) -> bool:
        return self.direction == 'up'


# ═════════════ MAIN LOOP ══════════════════════════════════════════════ #
# prices.txt: rows=timesteps, cols=instruments
prices = np.loadtxt(Path("prices.txt"))        # shape (T, nInst)
T, nInst = prices.shape

trader = CausalPIPTrader()
gates  = [GateState() for _ in range(nInst)]

# storage arrays
pos_sugg   = np.zeros((T, nInst), int)
pos_real   = np.zeros((T, nInst), int)
allow_flag = np.zeros((T, nInst), bool)

pnl_sugg   = np.zeros((T, nInst), float)
pnl_real   = np.zeros((T, nInst), float)
cum_sugg   = np.zeros((T, nInst), float)
cum_real   = np.zeros((T, nInst), float)

prev_sugg = np.zeros(nInst, int)
prev_real = np.zeros(nInst, int)

for t in range(T):
    # 0. Gate sees realised PnL *to previous bar*
    if t > 0:
        for i in range(nInst):
            gates[i].update(cum_real[t-1, i])

    # 1. Gate decision for this bar
    allow = np.array([g.allow for g in gates], bool)
    allow_flag[t] = allow

    # 2. Trader suggestion (uses prices ≤ t)
    sugg = trader.Alg(prices[:t+1].T)
    pos_sugg[t] = sugg

    # 3. Real target position
    real = np.where(allow, sugg, 0)
    pos_real[t] = real

    # 4. PnL bookkeeping (from *prev* positions)
    if t > 0:
        diff = prices[t] - prices[t-1]

        # always-on (PIP realised PnL)
        trade_amt_s = np.abs(sugg - prev_sugg) * prices[t]
        pnl_sugg[t] = prev_sugg * diff - COMMISSION_RATE * trade_amt_s
        cum_sugg[t] = cum_sugg[t-1] + pnl_sugg[t]

        # gated
        trade_amt_r = np.abs(real - prev_real) * prices[t]
        pnl_real[t] = prev_real * diff - COMMISSION_RATE * trade_amt_r
        cum_real[t] = cum_real[t-1] + pnl_real[t]

    prev_sugg = sugg
    prev_real = real

print("Simulation complete.")


# ═════════════ PLOTS ══════════════════════════════════════════════════ #
times = np.arange(T)

for i in range(nInst):
    fig, (ax_price, ax_gated, ax_pip) = plt.subplots(
        3, 1, figsize=(10, 12), sharex=True,
        gridspec_kw={"height_ratios": [1, 1, 1]}
    )

    # PRICE panel
    ax_price.plot(times, prices[:, i], lw=1, color="black")
    ax_price.fill_between(
        times,
        prices[:, i].min(),
        prices[:, i].max(),
        where=allow_flag[:, i],
        color="green",
        alpha=0.18,
        step="mid"
    )
    ax_price.fill_between(
        times,
        prices[:, i].min(),
        prices[:, i].max(),
        where=~allow_flag[:, i],
        color="red",
        alpha=0.18,
        step="mid"
    )
    ax_price.set_ylabel("Price")

    # GATED PnL panel
    ax_gated.plot(
        times, cum_real[:, i],
        lw=1.2, color="black",
        label="Realised PnL (gated)"
    )
    ax_gated.axhline(0, lw=0.5)
    ax_gated.set_ylabel("Gated PnL")
    ax_gated.legend(frameon=False, fontsize="small")

    # PIP realised cumulative PnL panel
    ax_pip.plot(
        times, cum_sugg[:, i],
        lw=1.2, ls="--",
        label="PIP Realised Cumulative PnL"
    )
    ax_pip.axhline(0, lw=0.5)
    ax_pip.set_ylabel("PIP PnL")
    ax_pip.set_xlabel("Timestep")
    ax_pip.legend(frameon=False, fontsize="small")

    fig.suptitle(f"Instrument {i:02d} — Gate-Aware & PIP PnL (1-bar delay)")
    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    plt.show()


In [ ]:
#!/usr/bin/env python3
"""
hybrid_pip_trader_gate_plot.py  (v4)
────────────────────────────────────
• Causal PIP trader  →  suggests positions every bar.
• Causal PnL gate    →  decides (with a 1-bar delay) whether trading is
  enabled (“GREEN”) or disabled (“RED”) based on realised cumulative PnL.
• PIP realised cumulative PnL panel added with PIP markers.

Trading rule
------------
real_pos[t] = suggested_pos[t]   if  gate says GREEN at bar t
            = 0                 otherwise

Key fixes vs previous version
-----------------------------
1. Gate is updated with `cum_real[t-1]` **before** it decides for bar *t*  
   → only a **one-bar** lag, not two.
2. Confidence panel removed.
3. Price shading uses the gate flag, not the suggested position.
4. **PIP realised cumulative PnL** is now plotted in its own panel.
5. **PIP markers** are now added on the PIP pnl plot (fully causal detection).
"""

from pathlib import Path
from typing import List, Optional, Tuple
import numpy as np
import matplotlib.pyplot as plt

# ═════════════ PIP DETECTION HELPERS (causal) ════════════════════════════ #
def _safe_rel_move(px: float, last_ext: float, eps: float = 1e-8) -> float:
    denom = max(abs(last_ext), eps)
    return (px - last_ext) / denom


def find_pips(series: np.ndarray, threshold: float) -> List[Tuple[int, float]]:
    """
    Causal zig‑zag pivot finder. Returns list of (idx, value).
    Marks a pivot as soon as the swing from last extreme exceeds threshold.
    """
    n = len(series)
    pips: List[Tuple[int, float]] = [(0, series[0])]
    last_ext, direction = series[0], None

    for i in range(1, n):
        move = _safe_rel_move(series[i], last_ext)

        if direction is None:
            if abs(move) >= threshold:
                direction = 'up' if move > 0 else 'down'
                last_ext = series[i]
                pips.append((i, series[i]))
        else:
            # extend current extreme
            if direction == 'up' and series[i] > last_ext:
                last_ext = series[i]
            elif direction == 'down' and series[i] < last_ext:
                last_ext = series[i]
            # reversal?
            rev = (_safe_rel_move(series[i], last_ext)
                   if direction == 'down'
                   else -_safe_rel_move(series[i], last_ext))
            if rev >= threshold:
                pips.append((i, last_ext))
                direction = 'down' if direction == 'up' else 'up'
                last_ext = series[i]

    # ensure we mark the final bar
    if pips[-1][0] != n - 1:
        pips.append((n - 1, series[-1]))
    return pips

# ═════════════ STRATEGY PARAMETERS ════════════════════════════════════ #
BEST_N = [
    200,200,400,200,400,400,100,300,100,200,
    100,500,100,300,200,100,200,100,200,300,
    100,200,500,100,400,100,100,400,100,100,
    400,500,300,200,400,500,100,400,100,100,
    100,300,400,500,100,400,500,100,500,300
]
BEST_W = [
    10,10,100,10,100,10,100,25,10,100,
    100,50,25,10,50,25,10,25,50,100,
    100,10,10,50,25,25,10,50,25,100,
    25,10,100,50,25,10,50,100,50,25,
    25,10,100,100,50,50,10,25,25,25
]

POSITION_LIMIT  = 10_000
COMMISSION_RATE = 0.0005
THR_MIN, THR_MAX, THR_STEP = 0.0, 0.700, 0.0005
THR_GRID = np.arange(THR_MIN, THR_MAX + THR_STEP, THR_STEP)

GATE_THR = 0.10  # 10% swing on cum-PnL flips gate
EPS = 1e-8

# ═════════════ LOW-LEVEL HELPERS / TRADER / GATE ═══════════════════════ #
def safe_rel_move(a: float, b: float, eps: float = EPS) -> float:
    return (a - b) / max(abs(b), eps)


def pip_update(px: float, last_ext: float, direction: Optional[int], thr: float) -> Tuple[int, float]:
    if direction == 0 or direction is None:
        mv = (px - last_ext) / last_ext
        if abs(mv) >= thr:
            return (1 if mv > 0 else -1), px
        return 0, last_ext
    if direction == 1:
        if px > last_ext:
            return 1, px
        if (last_ext - px) / last_ext >= thr:
            return -1, px
        return 1, last_ext
    if px < last_ext:
        return -1, px
    if (px - last_ext) / last_ext >= thr:
        return 1, px
    return -1, last_ext


class CausalPIPTrader:
    class _S:
        __slots__ = ("n", "w", "next_calib", "thr", "dir", "last_ext")
        def __init__(self, n: int, w: int):
            self.n, self.w = n, w
            self.next_calib = n
            self.thr = None
            self.dir = 0
            self.last_ext = None

    def __init__(self):
        self._st = [self._S(n, w) for n, w in zip(BEST_N, BEST_W)]

    def _best_thr(self, window: np.ndarray) -> float:
        best_thr, best_pnl = THR_GRID[0], -np.inf
        for thr in THR_GRID:
            dvec = np.zeros(len(window), np.int8)
            le, d = window[0], 0
            for k in range(1, len(window)):
                d, le = pip_update(window[k], le, d, thr)
                dvec[k] = d
            pnl = np.sum(dvec[:-1].astype(np.int32) * POSITION_LIMIT * np.diff(window))
            if pnl > best_pnl:
                best_pnl, best_thr = pnl, thr
        return best_thr

    def Alg(self, prc_so_far: np.ndarray) -> np.ndarray:
        nInst, t_seen = prc_so_far.shape
        pos = np.zeros(nInst, np.int32)
        for i in range(nInst):
            st, px = self._st[i], prc_so_far[i, -1]
            if st.last_ext is None:
                st.last_ext = px
            if t_seen >= st.next_calib:
                st.thr = self._best_thr(prc_so_far[i, t_seen - st.n:t_seen])
                st.next_calib += st.w
            if st.thr is None:
                continue
            new_dir, new_ext = pip_update(px, st.last_ext, st.dir, st.thr)
            st.dir, st.last_ext = new_dir, new_ext
            pos[i] = new_dir * POSITION_LIMIT
        return pos


class GateState:
    __slots__ = ("direction", "last_ext")
    def __init__(self):
        self.direction: Optional[str] = None
        self.last_ext = 0.0

    def update(self, pnl_now: float) -> None:
        mv = safe_rel_move(pnl_now, self.last_ext)
        if self.direction is None:
            if abs(mv) >= GATE_THR:
                self.direction = 'up' if mv > 0 else 'down'
                self.last_ext = pnl_now
            return
        if self.direction == 'up':
            if pnl_now > self.last_ext:
                self.last_ext = pnl_now
            elif safe_rel_move(pnl_now, self.last_ext) <= -GATE_THR:
                self.direction, self.last_ext = 'down', pnl_now
        else:
            if pnl_now < self.last_ext:
                self.last_ext = pnl_now
            elif safe_rel_move(pnl_now, self.last_ext) >= GATE_THR:
                self.direction, self.last_ext = 'up', pnl_now

    @property
    def allow(self) -> bool:
        return self.direction == 'up'


# ═════════════ MAIN LOOP: SIMULATION ═══════════════════════════════════ #
prices = np.loadtxt(Path("prices.txt"))
T, nInst = prices.shape
trader = CausalPIPTrader()
gates = [GateState() for _ in range(nInst)]

pos_sugg = np.zeros((T, nInst), int)
pos_real = np.zeros((T, nInst), int)
allow_flag = np.zeros((T, nInst), bool)

pnl_sugg = np.zeros((T, nInst), float)
cum_sugg = np.zeros((T, nInst), float)

pnl_real = np.zeros((T, nInst), float)
cum_real = np.zeros((T, nInst), float)

prev_sugg = np.zeros(nInst, int)
prev_real = np.zeros(nInst, int)

for t in range(T):
    if t > 0:
        for i in range(nInst):
            gates[i].update(cum_real[t-1, i])

    allow_flag[t] = np.array([g.allow for g in gates], bool)
    sugg = trader.Alg(prices[:t+1].T)
    pos_sugg[t] = sugg
    pos_real[t] = np.where(allow_flag[t], sugg, 0)

    if t > 0:
        diff = prices[t] - prices[t-1]
        trade_s = np.abs(sugg - prev_sugg) * prices[t]
        pnl_sugg[t] = prev_sugg * diff - COMMISSION_RATE * trade_s
        cum_sugg[t] = cum_sugg[t-1] + pnl_sugg[t]

        trade_r = np.abs(pos_real[t] - prev_real) * prices[t]
        pnl_real[t] = prev_real * diff - COMMISSION_RATE * trade_r
        cum_real[t] = cum_real[t-1] + pnl_real[t]

    prev_sugg, prev_real = sugg, pos_real[t]

print("Simulation complete.")


# ═════════════ PLOTS WITH PIP MARKERS ═══════════════════════════════════ #
pip_threshold = 0.3

times = np.arange(T)
for i in range(nInst):
    pivots = find_pips(cum_sugg[:, i], pip_threshold)
    idxs, vals = zip(*pivots)

    fig, (ax1, ax2, ax3) = plt.subplots(
        3, 1, sharex=True, figsize=(10, 12),
        gridspec_kw={'height_ratios': [1, 1, 1]}
    )

    # Price + gate shading
    ax1.plot(times, prices[:, i], color='black', lw=1)
    ax1.fill_between(
        times,
        prices[:, i].min(), prices[:, i].max(),
        where=allow_flag[:, i], color='green', alpha=0.18, step='mid'
    )
    ax1.fill_between(
        times,
        prices[:, i].min(), prices[:, i].max(),
        where=~allow_flag[:, i], color='red', alpha=0.18, step='mid'
    )
    ax1.set_ylabel('Price')

    # Gated PnL
    ax2.plot(
        times, cum_real[:, i], lw=1.2, color='black', label='Realised PnL (gated)'
    )
    ax2.axhline(0, lw=0.5)
    ax2.set_ylabel('Gated PnL')
    ax2.legend(frameon=False, fontsize='small')

    # PIP PnL + markers
    ax3.plot(
        times, cum_sugg[:, i], lw=1.2, ls='--', label='PIP Realised Cumulative PnL'
    )
    ax3.scatter(idxs, vals, s=50, color='blue', label='PIPs')
    ax3.axhline(0, lw=0.5)
    ax3.set_ylabel('PIP PnL')
    ax3.set_xlabel('Timestep')
    ax3.legend(frameon=False, fontsize='small')

    fig.suptitle(f'Instrument {i:02d} — Gate‑Aware & PIP PnL with PIPs')
    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    plt.show()
